# Progetto data Analytics

## Installing libraries

In [1]:
#for install libraries
!sudo apt install build-essential python-dev libxml2 libxml2-dev zlib1g-dev
!sudo pip install cairocffi
!pip install python-igraph

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
python-dev is already the newest version (2.7.15~rc1-1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libxml2 is already the newest version (2.9.4+dfsg1-6.1ubuntu1.2).
libxml2 set to manually installed.
libxml2-dev is already the newest version (2.9.4+dfsg1-6.1ubuntu1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
     |████████████████████████████████| 71kB 4.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/e7/5d/6f/fc3c2364dfd3c4cfd15d786b156077c52209d9af45496fdf12
Successfully built cairocffi
     |████████████████████████████████| 378kB 5.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/41/d6/02/34eeba

## Import Libraries and set local path

In [1]:
import networkx as nx
import igraph as ig
import louvain
from collections import OrderedDict
import numpy as np
import csv
from tqdm import tqdm_notebook as tqdm
import collections
import os 

baseURL = '../'

# Creation of a dictionaty for maintain all nets
Create a dictionary of lists, the key rappresents the day (start 0 end 29), the list is a list of nets about the specific day with this order: (attackGraph, messageGraph, TradeGraph, allianceGraph, allGraph, compactallGraph)

In [2]:
# our structure 
allNetwork = dict()

nGraph = 30

# for structure graphs inserting
attackGraph = ig.Graph(directed=True)
messageGraph = ig.Graph(directed=True)
tradeGraph = ig.Graph(directed=True)
totalGraph = ig.Graph(directed=True)
allianceGraph = ig.Graph(directed=True)
compactAllGraph = ig.Graph(directed=True)

for i in tqdm(range(0, nGraph)):
    # print("Day: ", (i + 1))
    
    # read all needed graphs
    attackGraph = attackGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/AttackGraph/cAttack-2009-12-"+str(i+1)+".graphml")
    messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/MessageGraph/cMessage-2009-12-"+str(i+1)+".graphml")
    tradeGraph = tradeGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/TradeGraph/cTrade-2009-12-"+str(i+1)+".graphml")
    allianceGraph = allianceGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/AllianceGraph/cAlliance-2009-12-"+str(i+1)+".graphml")
    totalGraph = totalGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/TotalGraph/cTotal-2009-12-"+str(i+1)+".graphml")
    
    # put as a list in a dictionary 
    allNetwork[i] = (attackGraph, messageGraph, tradeGraph, allianceGraph, totalGraph) 

# This cell is a usefull script for check user activity by user id

In [3]:
communityContIndegree = 0
for i in range(0, len(allNetwork)):
    for vertex in allNetwork[i][1].vs:
        print(vertex)
        if vertex['id'] == '4245':
            print("{:>20}{:>3}".format("PRESENT IN THE DAY:", str(i + 1)))
            print("{:>20}{:>3}".format("DAY INDEGREE: ", vertex.indegree()))
            print("{:>20}{:>3}".format("DAY OUTDEGREE: ", vertex.outdegree()))     
            # print("{:>20}{:>3}".format("COMMUNITY: ", vertex['Community'])) 
            for edge in allNetwork[i][1].es:
                if allNetwork[i][3].vs[edge.tuple[0]]['id'] == '4245':
                    #print("{:>20}{:>3}".format("OUTDEGREE LABEL: ", edge['Label']))
                    #print("{:>20}{:>3}".format("QUANTITY: ", edge['Weight']))
                    print("IN EDGE: ", edge)
                if allNetwork[i][1].vs[edge.tuple[1]]['id'] == '4245':
                    print("OUT EDGE: ", edge)
                    # print("{:>20}{:>3}".format("INDEGREE LABEL: ", edge['Label']))
                    # print("{:>20}{:>3}".format("QUANTITY: ", edge['Weight']))
            print("") 

igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 0, {'id': '8603'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 1, {'id': '282'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 2, {'id': '3621'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 3, {'id': '2370'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 4, {'id': '9462'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 5, {'id': '3377'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 6, {'id': '2753'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 7, {'id': '3446'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 8, {'id': '1480'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 9, {'id': '2240'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 10, {'id': '3237'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 11, {'id': '9152'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 12, {'id': '8705'})
igraph.Vertex(<igraph.Graph object at 0x114a5b408>, 13, {'id': '6179'})
igr

igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 148, {'id': '438'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 149, {'id': '983'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 150, {'id': '539'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 151, {'id': '56'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 152, {'id': '9126'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 153, {'id': '2277'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 154, {'id': '8331'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 155, {'id': '8136'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 156, {'id': '1120'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 157, {'id': '10064'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 158, {'id': '2758'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 159, {'id': '5577'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 160, {'id': '4803'})
igraph.Vertex(<igraph.Graph object at 0x114cfe9a8>, 161

KeyError: 'Attribute does not exist'

# START ANALYSIS !!!!

### Users number trend in the 30 days for total net

In [3]:
def userNumberTrend(allNetwork):
    nUsers = [0]*len(allNetwork)
    for i in range(0, len(allNetwork)):
        nUsers[i] = allNetwork[i][4].vcount()

    print("Users in the 30 days: ", nUsers)
    print("")
    print("{:>53}{:>7}.".format("Average users number in the 30 days is: ", round(sum(nUsers)/len(nUsers)), 2))
    print("{:>53}{:>7}.".format("The maximum number of users was recorded on the day: ", nUsers.index(max(nUsers))+1))
    print("")
    return nUsers

### User number trend in alliance graph (sign in community file, present in network and active in the community) 

In [4]:
def userCommunityNumberTrend(communityStory, communityNumber):
    nActiveUser = list()
    contActiveUser = 0
    community = communityStory[0].vs[0]['Community']
    
    with open(baseURL+'Dash/SingleCommunityStudy/CSVFILE/nUserTrend.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'totalSign', 'presentInNet', 'activeInCommunity'])
        
        for day in communityStory:
            for vertex in communityStory[day].vs:
                if vertex.outdegree() > 0:
                    contActiveUser += 1
            nActiveUser.append(contActiveUser)
            contActiveUser = 0
        
        for i in range(0, len(communityStory)):
            filewriter.writerow([i+1, communityNumber[i][community][1], len(communityNumber[i][community][0]),  nActiveUser[i]])
        
    print("")

### Edges number trend in the 30 days

In [5]:
def edgeNumberTrend(allNetwork):  
    nAttack = [0]*len(allNetwork)
    nTrade = [0]*len(allNetwork)
    nMessage = [0]*len(allNetwork)

    for i in range(0, nGraph):
        for edge in allNetwork[i][4].es:
            if edge['Label'] == "Attack":
                nAttack[i] += edge['Weight']
            elif edge['Label'] == "Trade":
                nTrade[i] += edge['Weight']
            elif edge['Label'] == "Message":
                nMessage[i] += edge['Weight']
            else:
                print("This is a problem...")

    print("{:>33}".format("Number of attacks in the 30 days:"), end = " ")
    print(nAttack)
    print("")
    print("{:>32}".format("Number of trades in the 30 days:"), end = " ")
    print(nTrade)
    print("")
    print("{:>34}".format("Number of messages in the 30 days:"), end = " ")
    print(nMessage)
    print("")
    print("{:>40}{:>9}".format("Average number of attacks in 30 days:", sum(nAttack)/len(nAttack)))
    print("{:>40}{:>9}".format("Average number of trades a in 30 days:", round(sum(nTrade)/len(nTrade), 1)))
    print("{:>40}{:>9}".format("Average number of messages a in 30 days:", round(sum(nMessage)/len(nMessage), 1)))  
    print("")
    print("{:>40}{:>9}".format("Day with the highest number of attacks:", nAttack.index(max(nAttack))+1))
    print("{:>40}{:>9}".format("Day with the highest number of trades:", nTrade.index(max(nTrade))+1))
    print("{:>40}{:>9}".format("Day with the highest number of messages:", nMessage.index(max(nMessage))+1)) 
    print("")
    
    return nAttack, nTrade, nMessage

### Edges number trend for specific community

In [6]:
def edgeCommunityNumberTrend(communityStory):
    nTrade = [0]*len(communityStory)
    nMessage = [0]*len(communityStory)
    
    with open(baseURL+'Dash/SingleCommunityStudy/CSVFILE/nEdgeTrend.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'type', 'quantity'])
    
        for i in range(0, len(communityStory)):
            for edge in communityStory[i].es:
                nTrade[i] += edge['TWeight']
                nMessage[i] += edge['MWeight']
            filewriter.writerow([i+1, 'nMessage', nMessage[i]])
            filewriter.writerow([i+1, 'nTrade', nTrade[i]])
                
    print("{:>32}".format("Number of trades in the 30 days:"), end = " ")
    print(nTrade)
    print("")
    print("{:>34}".format("Number of messages in the 30 days:"), end = " ")
    print(nMessage)
    print("")
    print("{:>40}{:>9}".format("Average number of trades a in 30 days:", round(sum(nTrade)/len(nTrade), 1)))
    print("{:>40}{:>9}".format("Average number of messages a in 30 days:", round(sum(nMessage)/len(nMessage), 1)))  
    print("")
    print("{:>40}{:>9}".format("Day with the highest number of trades:", nTrade.index(max(nTrade))+1))
    print("{:>40}{:>9}".format("Day with the highest number of messages:", nMessage.index(max(nMessage))+1)) 
    print("")  

### Compute Clustering, Density, Reciprocity for alliance graph (nodes labeled with community id) and get the number of edges  from it. We retrieve data from extra-community and intra-community relationship (trade e message)

In [7]:
def computeStaticOnTotal(allNetwork):
    
    allDayMGraph, allDayTGraph, allDayMGraphNoSame, allDayTGraphNoSame, allDayMGraphSame, allDayTGraphSame = retrieveGraph(allNetwork)

    onlyAlliance = dict()
    for day in allNetwork:
        onlyAlliance[day] = allNetwork[day][3]
    
    for day in range(0, 30):
        if len(allNetwork[day][3].vs()) == len(allDayMGraph[day].vs()) and len(allDayMGraph[day].vs()) == len(allDayTGraph[day].vs()):
            continue
        else:
            print("PROBLEM TO DAY: ", day + 1)
    
    computeClusteringTotalCoefficient(onlyAlliance, '')
    computeClusteringTotalCoefficient(allDayMGraph, 'M')
    computeClusteringTotalCoefficient(allDayTGraph, 'T')

    computeClusteringTotalCoefficient(allDayMGraphNoSame, 'N_M')
    computeClusteringTotalCoefficient(allDayTGraphNoSame, 'N_T')
    
    computeClusteringTotalCoefficient(allDayMGraphSame, 'S_M')
    computeClusteringTotalCoefficient(allDayTGraphSame, 'S_T')
    print("CLUSTERING COEFFICIENT ANALYSIS DONE")

    computeTotalDensity(onlyAlliance, '')
    computeTotalDensity(allDayMGraph, 'M')
    computeTotalDensity(allDayTGraph, 'T')
    
    computeTotalDensity(allDayMGraphNoSame, 'N_M')
    computeTotalDensity(allDayTGraphNoSame, 'N_T')
    
    computeTotalDensity(allDayMGraphSame, 'S_M')
    computeTotalDensity(allDayTGraphSame, 'S_T')
    print("DENSITY ANALYSIS DONE")

    computeTotalReciprocity(onlyAlliance, '')
    computeTotalReciprocity(allDayMGraph, 'M')
    computeTotalReciprocity(allDayTGraph, 'T')
    
    computeTotalReciprocity(allDayMGraphNoSame, 'N_M')
    computeTotalReciprocity(allDayTGraphNoSame, 'N_T')
    
    computeTotalReciprocity(allDayMGraphSame, 'S_M')
    computeTotalReciprocity(allDayTGraphSame, 'S_T')
    print("RECIPROCITY ANALYSIS DONE") 
    
    getNEdges(allDayMGraph, "TotalMessage")
    getNEdges(allDayTGraph, "TotalTrade")
    
    getNEdges(allDayMGraphNoSame, "NoSameMessage")
    getNEdges(allDayTGraphNoSame, "NoSameTrade")
    
    getNEdges(allDayMGraphSame, "SameMessage")
    getNEdges(allDayTGraphSame, "SameTrade")
    print("NUMBER EDGES ANALYSIS DONE")
    
    computeTotalDiameter(onlyAlliance, '')
    computeTotalDiameter(allDayMGraph, 'M')
    computeTotalDiameter(allDayTGraph, 'T')
    
    computeTotalDiameter(allDayMGraphNoSame, 'N_M')
    computeTotalDiameter(allDayTGraphNoSame, 'N_T')
    
    computeTotalDiameter(allDayMGraphSame, 'S_M')
    computeTotalDiameter(allDayTGraphSame, 'S_T')
    print("DIAMETER ANALYSIS DONE") 
    
    computeTotalAveragePathLength(onlyAlliance, '')
    computeTotalAveragePathLength(allDayMGraph, 'M')
    computeTotalAveragePathLength(allDayTGraph, 'T')
    
    computeTotalAveragePathLength(allDayMGraphNoSame, 'N_M')
    computeTotalAveragePathLength(allDayTGraphNoSame, 'N_T')
    
    computeTotalAveragePathLength(allDayMGraphSame, 'S_M')
    computeTotalAveragePathLength(allDayTGraphSame, 'S_T')
    print("DIAMETER ANALYSIS DONE")
    
    totalAllianceInDegreeDistribution(allDayMGraphNoSame, 'N_M')
    totalAllianceInDegreeDistribution(allDayTGraphNoSame, 'N_T')
    
    totalAllianceInDegreeDistribution(allDayMGraphSame, 'S_M')
    totalAllianceInDegreeDistribution(allDayTGraphSame, 'S_T')
    
    totalAllianceOutDegreeDistribution(allDayMGraphNoSame, 'N_M')
    totalAllianceOutDegreeDistribution(allDayTGraphNoSame, 'N_T')
    
    totalAllianceOutDegreeDistribution(allDayMGraphSame, 'S_M')
    totalAllianceOutDegreeDistribution(allDayTGraphSame, 'S_T')
    
def totalAllianceInDegreeDistribution(allDayGraph, dayType):
    
    dayInDistribution = dict()
    
    alreadyManage = list()
    
    nZeroIndegree = 0
    
    nIn = 0
    
    for day in allDayGraph:
        # print("DAY: ", day + 1)
        # print(allDayGraph[day].summary())
        dayInDistribution[0] = 0
        
        for vertex in allDayGraph[day].vs:
            if vertex.indegree() == 0 and vertex.outdegree() == 0:
                nZeroIndegree += 1
                continue 
            for edge in allDayGraph[day].es:
                if allDayGraph[day].vs[edge.tuple[1]]['label'] == vertex['label']:
                    nIn += edge['Weight']

            if nIn in dayInDistribution:
                dayInDistribution[nIn] += 1
            else:
                dayInDistribution[nIn] = 1
                
            nIn = 0
        
        dayInDistribution[0] += nZeroIndegree
    
        nNodesCheck = 0
        for key in dayInDistribution:
            nNodesCheck += dayInDistribution[key]
        # print("NODES CHECK ON DAY DISTRIBUTION: ", nNodesCheck)
        
        if day == 0 and dayType == 'N_M':
            with open(baseURL+"Dash/AllianceGraph/indegreeDistribution.csv", 'a+') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
                
                filewriter.writerow(['day', 'type', 'range', 'value'])
                
                
        writeTotalDataInFile(dayInDistribution, dayType, (len(allDayGraph[day].vs)), day, 'in')
      
        dayInDistribution = dict()
        nZeroIndegree = 0

        
def totalAllianceOutDegreeDistribution(allDayGraph, dayType):
    
    dayOutDistribution = dict()
    
    alreadyManage = list()
    
    nZeroOutdegree = 0
    
    nOut = 0
    
    for day in allDayGraph:
        # print("DAY: ", day + 1)
        # print(allDayGraph[day].summary())
        dayOutDistribution[0] = 0
        
        for vertex in allDayGraph[day].vs:
            if vertex.indegree() == 0 and vertex.outdegree() == 0:
                nZeroOutdegree += 1
                continue 
            for edge in allDayGraph[day].es:
                if allDayGraph[day].vs[edge.tuple[0]]['label'] == vertex['label']:
                    nOut += edge['Weight']

            if nOut in dayOutDistribution:
                dayOutDistribution[nOut] += 1
            else:
                dayOutDistribution[nOut] = 1
                
            nOut = 0
        
        dayOutDistribution[0] += nZeroOutdegree
    
        nNodesCheck = 0
        for key in dayOutDistribution:
            nNodesCheck += dayOutDistribution[key]
        # print("NODES CHECK ON DAY DISTRIBUTION: ", nNodesCheck)
        
        if day == 0 and dayType == 'N_M':
            with open(baseURL+"Dash/AllianceGraph/outdegreeDistribution.csv", 'a+') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
                
                filewriter.writerow(['day', 'type', 'range', 'value'])
                
                
        writeTotalDataInFile(dayOutDistribution, dayType, (len(allDayGraph[day].vs)), day, 'out')
      
        dayOutDistribution = dict()
        nZeroOutdegree = 0
                
def writeTotalDataInFile(dayDistribution, typeOfDistribution, nNodes, day, typeEdge):
    
        finalDayDistribution = dict()
    
        finalDayDistribution[0] = 0
        finalDayDistribution['1-10'] = 0
        finalDayDistribution['11-20'] = 0
        finalDayDistribution['21-30'] = 0
        finalDayDistribution['31-40'] = 0
        finalDayDistribution['41-50'] = 0
        finalDayDistribution['51-60'] = 0
        finalDayDistribution['61-70'] = 0
        finalDayDistribution['71-80'] = 0
        finalDayDistribution['81-90'] = 0
        finalDayDistribution['91-100'] = 0
        finalDayDistribution['>100'] = 0 
        
        startBound = 1
        endBound = 10
        nEdges = 0
    
        finalDayDistribution[0] = dayDistribution[0]
    
        while endBound <= 100:
            for key in dayDistribution:
                nEdges += key * dayDistribution[key]
                if key > 100:
                    finalDayDistribution[">100"] += dayDistribution[key]
                elif key >= startBound and key <= endBound:
                    finalDayDistribution[""+str(startBound)+"-"+str(endBound)+""] += dayDistribution[key]
            endBound += 10
            startBound += 10
        
        finalDayDistribution[">100"] = finalDayDistribution[">100"]/10
        nEdges = nEdges/10
        nodesCont = 0
        totalDistribution = 0
        
        for key in finalDayDistribution:
            nodesCont += int(finalDayDistribution[key])
            # print(key, finalDayDistribution[key])
            totalDistribution += finalDayDistribution[key]/nNodes
            
        # print(nodesCont)
        # print(totalDistribution)
        # print(typeOfDistribution, ": ", nEdges)
        # print("")
        
        with open(baseURL+"Dash/AllianceGraph/"+str(typeEdge)+"degreeDistribution.csv", 'a+') as csvfile:
            filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)

            for key in finalDayDistribution:
                value = round(finalDayDistribution[key]/nNodes, 9)

                filewriter.writerow([day + 1, typeOfDistribution, key, value])
        
        if typeEdge == 'in':
            with open(baseURL+"Dash/AllianceGraph/average"+str(typeOfDistribution)+"Degree.csv", 'a+') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            
                if day == 0:
                    filewriter.writerow(["day", "averageDegree"])
        
                filewriter.writerow([day + 1, round(nEdges/nNodes+(0.00001), 9)])
    
def computeTotalAveragePathLength(allDayGraph, dayType):
    
    dayAveragePathLength = list()
    
    for day in allDayGraph:
        dayAveragePathLength.append(allDayGraph[day].average_path_length(directed = True, unconn = True))
        
    with open(baseURL+'Dash/AllianceGraph/AP_average'+dayType+'PathLengthData.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'averagePathLength'])
        
        for day in range(0, len(dayAveragePathLength)):
            filewriter.writerow([day+1, dayAveragePathLength[day]])
        
def computeTotalDiameter(allDayGraph, dayType):
    
    dayDiameter = list()
    
    for day in allDayGraph:
        dayDiameter.append(allDayGraph[day].diameter(directed=True, unconn=True))
        
    with open(baseURL+'Dash/AllianceGraph/D_diameter'+dayType+'Data.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'diameter'])
        
        for day in range(0, len(dayDiameter)):
            filewriter.writerow([day+1, dayDiameter[day]])       

def computeTotalDensity(allDayGraph, dayType):
    
    dayDensity = list()
    
    for day in allDayGraph:
        dayDensity.append(allDayGraph[day].density(loops = False))
        
    with open(baseURL+'Dash/Density/DE_density'+dayType+'Data.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'density'])
        
        for day in range(0, len(dayDensity)):
            filewriter.writerow([day + 1, dayDensity[day]])


def computeTotalReciprocity(allDayGraph, dayType):
    
    dayReciprocity = list()
    
     #reciprocity(ignore_loops=True, mode="default")
    
    for day in allDayGraph:
        dayReciprocity.append(allDayGraph[day].reciprocity(ignore_loops=True, mode=1))
        
    with open(baseURL+'Dash/Reciprocity/RE_reciprocity'+dayType+'Data.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'Reciprocity'])
        
        for day in range(0, len(dayReciprocity)):
            filewriter.writerow([day+1, dayReciprocity[day]]) 

def computeClusteringTotalCoefficient(allDayGraph, dayType):
    
    dayClusteringCoefficient = list()

    for day in allDayGraph:
        dayClusteringCoefficient.append(allDayGraph[day].transitivity_undirected())
        
    with open(baseURL+'Dash/ClusteringCoefficient/CC_clustering'+dayType+'CoefficientData.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'clusteringCoefficient'])
        
        for day in range(0, len(dayClusteringCoefficient)):
            filewriter.writerow([day+1, dayClusteringCoefficient[day]])
            
def getNEdges(allDayGraph, dayType):
    
    nEdges = list()
    weigthsEdges = dict()
    totalDayEdges = 0
    
    for day in allDayGraph:
        totalDayEdges = 0
        nEdges.append(len(allDayGraph[day].es()))
        for edge in allDayGraph[day].es():
            totalDayEdges += edge['Weight']
        weigthsEdges[day] = totalDayEdges
            
        
    with open(baseURL+'Dash/EdgesInAllianceGraph/'+dayType+'_NEdges.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'nEdges', 'TotalWeight'])
        
        for day in range(0, len(nEdges)):
            filewriter.writerow([day+1, nEdges[day], weigthsEdges[day]])


### Trickster detection tool 
This tool is designed for identifying dishonest users who manage two accounts and use one for the main account. The assumption is: if i see you two times do more trading than any else action you are a suspected and is necessary to see what are you done in the period (30 days). 

In [8]:
# Main function call all function that compose the tool
def tricksterAnalysis(allNetwork):
    
    # make a list with main suspect for every day, in the end we have a following structure
    #{DAY: {idSuspected: {idBeneficiry: qta}}} 
    # a dictionary where for first external key we have a day, inside 1 lvl key is suspected id, in the last 2 lvl
    # a dictionary of beneficiary key with quantity of trading
    dailySuspicions = dict()
    print("Start trickester Analysis")
    print("Watching for dishonest user in the period...")
    for i in tqdm(range(0, len(allNetwork))):
        # print("DAY: ", str(i+1))
        dailySuspicions[i] = tricksterDetectionTool(allNetwork[i][4])
        
    if os.path.exists(baseURL+'Dash/TricksterDetection/tricksterData.csv'):
        if os.path.exists(baseURL+'Dash/TricksterDetection/tricksterActivityDay.csv'):
                os.remove(baseURL+'Dash/TricksterDetection/tricksterData.csv')
                os.remove(baseURL+'Dash/TricksterDetection/tricksterActivityDay.csv')
        
    with open(baseURL+'Dash/TricksterDetection/tricksterData.csv', 'a') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['Suspect', 'GuiltyIndex', 'TradeIn', 'MessageIn', 'AttackIn', 'TradeOut', 'MessageOut', 'AttackOut', 'Beneficiary', 'ReceivedFrom'])
    
    with open(baseURL+'Dash/TricksterDetection/TmptricksterData.csv', 'a') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['Suspect', 'GuiltyIndex', 'TradeIn', 'MessageIn', 'AttackIn', 'TradeOut', 'MessageOut', 'AttackOut', 'Beneficiary', 'ReceivedFrom'])
        
    with open(baseURL+'Dash/TricksterDetection/tricksterActivityDay.csv', 'a') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['Day', 'Suspect', 'Type', 'Present'])
    
    # from first skimming compute a guilty index and print the player with more then 0.3 on index
    edgesCount(printRealSuspected(dailySuspicions, len(dailySuspicions)), allNetwork)

# create a day dictionary structur (see below)
def tricksterDetectionTool(totalNet):
    zeroIndegreeVertex = list()

    # get, specific net, all vertex with 0 indegree
    for vertex in totalNet.vs:
        if vertex.indegree() == 0:
            zeroIndegreeVertex.append(vertex)
    
    '''for vertex in zeroIndegreeVertex:
        print(vertex)
        print(vertex.indegree())
        print(vertex.outdegree())'''
    # print(zeroIndegreeVertex)
    
    # create a dictionary for keeping suspects with respective beneficiary
    suspecteds = dict()
    beneficiary = dict()
    key = ""
    
    # for every zero indegree vertex count the number of edge and the number of trade edge
    # if find a trade edge save the beneficiary of this trade and quantity
    for vertex in zeroIndegreeVertex:
        nEdges = 0
        nTrades = 0
        key = ""
        beneficiary = dict()
        for edge in totalNet.es:
            if totalNet.vs[edge.tuple[0]]['id'] == vertex['id']:
                nEdges += edge['Weight']
                if edge['Label'] == 'Trade':
                #    if totalNet.vs[edge.tuple[0]]['id'] == vertex['id'] == "7855":
                #        print(edge)
                    # print(totalNet.vs[edge.tuple[0]]['id'])
                    # print(vertex['id'])
                    if totalNet.vs[edge.tuple[1]]['id'] in beneficiary:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] += edge['Weight']
                    else:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] = edge['Weight']
                    nTrades += edge['Weight']
        # if the trades edges number is more than 95%, the node is suspected, saved it!
        if nTrades/nEdges >= 0.95:
            # print(nEdges)
            # print(nTrades)
            key = str(vertex['id'])
            # print(key)
            # print(beneficiary)
            suspecteds[key] = beneficiary
    
    # for key in suspecteds:
    #    print(key, suspecteds[key])
    return suspecteds    

# from all day suspects we want a more compact structure for see what happen to our suspects node in the period
# (30 days). The structure is a dictionary: 
# {idSuspected: [nWatchTimes, {idBeneficiary: [nWatchTimes, qnt]}, nWatchSuspectAsBeneficiary, qnt]}
# from this we can delete from suspects list the node with only one appearance
def printRealSuspected(allSuspects, nDays):
    
    print("Compact recovered information...")
    beneficiaries = dict()
    suspectsSummary = dict()
    finalSuspects = dict()
    
    
    for day in allSuspects:
        for suspect in allSuspects[day]:
            # print("Suspected: ", suspect)
            # increase the number of nWatchTimes about suspects
            if str(suspect) in suspectsSummary:
                suspectsSummary[suspect][0] += 1
            else:
                suspectsSummary[suspect] = list([1, dict(), 0, 0])
                suspectsSummary[suspect][0] = 1
                # print(suspectsSummary[suspect][0])
            # print("TIME: ", suspectsSummary[suspect])
            
            # for all suspect beneficiary increase nWatchTimes about beneficiary and qnt about trade
            for beneficiary in allSuspects[day][suspect]:
                # print("Beneficiary is: ", beneficiary)
                # print("Benefit is: ", allSuspects[day][suspect][beneficiary])
                # print(str(beneficiary) in suspectsSummary[suspect][1])
                if str(beneficiary) in suspectsSummary[suspect][1]:
                    suspectsSummary[suspect][1][beneficiary][0] += 1
                    suspectsSummary[suspect][1][beneficiary][1] += allSuspects[day][suspect][beneficiary]
                else:
                    suspectsSummary[suspect][1][beneficiary] = [1, allSuspects[day][suspect][beneficiary]]

                    
            # print(suspectsSummary)

    # increase nWatchSuspectAsBeneficiary qnt index
    for day in allSuspects:
        for suspect in allSuspects[day]:
            for beneficiary in allSuspects[day][suspect]:
                if beneficiary in suspectsSummary:
                    # print("HERE MAN")
                    suspectsSummary[beneficiary][2] += 1
                    suspectsSummary[beneficiary][3] += allSuspects[day][suspect][beneficiary]
                    # print(suspectsSummary[beneficiary])
                    
            # print(suspectsSummary)
            # print("")
                
    # suspectsSummary = sorted(suspectsSummary.items(), key=lambda e: e[1][2])
    # order from the most observed to the least observed 
    suspectsSummary = OrderedDict(sorted(suspectsSummary.items(), key=lambda t: t[1][0], reverse=True))
    
    # print("Complete: ", len(suspectsSummary))
    # for suspect in suspectsSummary:
    #    print(suspect, suspectsSummary[suspect])
    
    # if suspect is observed only one time or is observed as many times as there are beneficiaries; 
    # is no longer a suspect
    for suspect in suspectsSummary:
        if suspectsSummary[suspect][0] > 1 and suspectsSummary[suspect][0] != suspectsSummary[suspect][2]:
            finalSuspects[suspect] = suspectsSummary[suspect]
            
    # print("With elimination: ", len(finalSuspects))
    # for suspect in finalSuspects:
        # print(suspect, finalSuspects[suspect])
    
    # print("")
    # print(finalSuspects)
    
    return finalSuspects
 
# compute a Guiltyindex (based to experimental observations) and print the node with more than .3
def edgesCount(finalSuspects, allNet):
    print("This are the main suspects!")
    edgesInCount = dict()
    edgesOutCount = dict()
    tradeEdgesInCount = dict()
    edgesOutPlayer = dict()
    finalResult = dict()
 
    activityDay = list()
    attackDay = list()
    tradeDay = list()
    
    inCount = 0
    outCount = 0
    tradeOutCount = 0
    tradeInCount = 0
    guiltyIndex = 0
    maxIndex = 0
    cont = 0
    
    # count number of edge about all typology
    for suspect in finalSuspects:
        
        activityDay.clear()
        attackDay.clear()
        tradeDay.clear()
        
        edgesInCount.clear()
        edgesOutCount.clear()
        edgesOutPlayer.clear()
        tradeEdgesInCount.clear()
        
        playerServed = ""
        fromPlayer = ""
        
        for i in range(0, 3):
            edgesInCount['Attack'] = 0
            edgesInCount['Trade'] = 0
            edgesInCount['Message'] = 0
            edgesOutCount['Attack'] = 0
            edgesOutCount['Trade'] = 0
            edgesOutCount['Message'] = 0
        
        inCount = 0
        outCount = 0
        tradeOutCount = 0
        tradeInCount = 0
        guiltyIndex = 0
        
        # append a activity day of suspect
        for day in allNet:
            for vertex in allNet[day][4].vs:
                if str(vertex['id']) == suspect:
                    activityDay.append(str(day+1))

            for edge in allNet[day][4].es:
                if allNetwork[day][4].vs[edge.tuple[0]]['id'] == suspect:
                    if edge['Label'] in edgesOutCount:
                        edgesOutCount[edge['Label']] += edge['Weight']
                    else:
                        edgesOutCount[edge['Label']] = edge['Weight']
                    if edge['Label'] == 'Trade':
                        if allNetwork[day][4].vs[edge.tuple[1]]['id'] in edgesOutPlayer:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] += edge['Weight']
                        else:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] = edge['Weight']
                        if (day + 1) not in tradeDay:
                            tradeDay.append(day + 1)
                if allNet[day][4].vs[edge.tuple[1]]['id'] == suspect:
                    if edge['Label'] == 'Trade':
                        if allNet[day][4].vs[edge.tuple[0]]['id'] in tradeEdgesInCount:
                            tradeEdgesInCount[allNetwork[day][4].vs[edge.tuple[0]]['id']] += edge['Weight']
                        else:
                            tradeEdgesInCount[allNetwork[day][4].vs[edge.tuple[0]]['id']] = edge['Weight']
                    if edge['Label'] in edgesInCount:
                        edgesInCount[edge['Label']] += edge['Weight']
                    else:
                        edgesInCount[edge['Label']] = edge['Weight']
                    if edge['Label'] == 'Attack':
                        if (day + 1) not in attackDay:
                            attackDay.append(day + 1)
        guiltyIndex = edgesOutCount['Trade']*((1/len(edgesOutPlayer)))*1.3
        # print("Trade out: ", edgesOutCount['Trade'])
        # print("People Served: ", len(edgesOutPlayer))
        # print(guiltyIndex)
        guiltyIndex -= (edgesInCount['Trade']*5)
        # print(guiltyIndex)
        guiltyIndex -= (edgesOutCount['Message'])*1
        # print(guiltyIndex)
        guiltyIndex += (edgesInCount['Message'])*0.3
        # print(guiltyIndex)
        guiltyIndex -= edgesOutCount['Attack']*0.4
        # print(guiltyIndex)
        guiltyIndex -= edgesInCount['Attack']*0.08
        # print(guiltyIndex)
        # print("DENOMINATOR: ", edgesInCount['Attack']+edgesInCount['Trade']+edgesInCount['Message']+edgesOutCount['Attack']+edgesOutCount['Message']+edgesOutCount['Trade'])
        guiltyIndex = guiltyIndex / (edgesInCount['Attack']*0.08+edgesInCount['Trade']+edgesInCount['Message']+edgesOutCount['Attack']+edgesOutCount['Message']*2+edgesOutCount['Trade'])
        for player in edgesOutPlayer:
            playerServed += player+":"+str(int(edgesOutPlayer[player]))+" "
        for player in tradeEdgesInCount:
            fromPlayer += player+":"+str(int(tradeEdgesInCount[player]))+" "
        if guiltyIndex >= 0.3:
            if guiltyIndex > maxIndex:
                maxIndex = guiltyIndex
            print("GULTY INDEX: ", round(guiltyIndex, 4))
            print("SUSPECT: ", suspect)
            print("EDGES IN: ", edgesInCount)
            print("EDGES OUT: ", edgesOutCount)
            print("PLAYER SERVED: ", edgesOutPlayer)
            print("ACTIVITY DAYS: ", activityDay)
            print("DAYS OF ATTACKS SUFFERED: ", attackDay)
            print("DAYS OF TRADE DONE: ", tradeDay)
            print("PLAYER TRADE WITH SUSPECT: ", tradeEdgesInCount)
            print(" ")
            
            with open(baseURL+'Dash/TricksterDetection/TmptricksterData.csv', 'a') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
                filewriter.writerow([suspect, round(guiltyIndex, 4), edgesInCount['Trade'], edgesInCount['Message'], edgesInCount['Attack'], edgesOutCount['Trade'], edgesOutCount['Message'], edgesOutCount['Attack'], playerServed, fromPlayer])
            with open(baseURL+'Dash/TricksterDetection/tricksterActivityDay.csv', 'a') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
                for j in range(0, 30):
                    if j+1 in tradeDay:
                        filewriter.writerow([j+1 , suspect, "Trade", 1])
                    else:
                        filewriter.writerow([j+1 , suspect, "Trade", 0])
                    
                    if j+1 in attackDay:
                        filewriter.writerow([j+1 , suspect, "Attack", 1])
                    else:
                        filewriter.writerow([j+1 , suspect, "Attack", 0])
                        
    with open(baseURL+'Dash/TricksterDetection/TmptricksterData.csv') as csvfile1:    
        reader = csv.reader(csvfile1, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        with open(baseURL+'Dash/TricksterDetection/tricksterData.csv', 'a') as csvfile2:  
            filewriter = csv.writer(csvfile2, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            cont = 0
            for line in reader:
                if cont == 0:
                    cont += 1
                    continue
                print(line)
                filewriter.writerow([line[0], float(line[1])/maxIndex, line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9]])
                
    
    os.remove(baseURL+'Dash/TricksterDetection/TmptricksterData.csv')        

## Communities Dynamics

#### Add communities label to vertex

In [9]:
def addCommunitiesLabel(allCompactNetwork, dayCommunity):
        
    communities = open(baseURL +  "Dataset/Communities/OrganizedCommunities/Ocommunities-2009-12-"+str(dayCommunity + 1)+".txt", "r")
    
    communityCont = 0
    nNodes = 0
    nNodesPresent = 0
    nDissolved = 0
    nActive = 0
    
    present = False
    
    communityAffiliates = dict()
    
    affiliatesNodes = list()
    noMatchNodes = list()
    nodesGraphNoMatch = list()
    vertexToDelete = list()
    
    for line in communities:
        # print("COMMUNITY NUMBER: ", cont)
        # print(line)
        line = line.split()
        # print(type(line[0]))
        for node in line:
            if node == "DISSOLVED":
                nDissolved += 1
                affiliatesNodes.append("DISSOLVED")
                continue
            present = False
            nNodes += 1
            # allCompactNetwork.vs.select('id' = node)['Community'] = communityCont
            for vertex in allCompactNetwork.vs:
                if vertex['id'] == str(node):
                    if len(line) > 10:
                        affiliatesNodes.append(vertex['id'])
                        vertex['Community'] = communityCont
                      #  print("VERTEX IS: ", vertex)
                        if vertex.outdegree() > 0:
                            nActive += 1
                    else:
                        affiliatesNodes.append(vertex['id'])
                        vertex['Community'] = -communityCont
                        if vertex.outdegree() > 0:
                            nActive += 1
                    nNodesPresent += 1
                    present = True
            if not present:
                noMatchNodes.append(node)
       
        communityAffiliates[communityCont] = list((list(affiliatesNodes), len(line), nActive))
        
        communityCont += 1
        affiliatesNodes.clear()
        nActive = 0
        
    # print("NUMBER OF NODES IN DAY TOTAL GRAPH: ", len(allCompactNetwork.vs))
    
    for vertex in allCompactNetwork.vs:
        if vertex['Community'] is None:
            nodesGraphNoMatch.append(vertex['id'])
            vertexToDelete.append(vertex.index)
            # print(vertex.index)
            # print(vertex)
    allCompactNetwork.delete_vertices(vertexToDelete)
        
    # print("NUMBER OF NODES IN COMMUNITIES FILE: ", nNodes)
    # print("NUMBER OF COMMUNITIES: ", communityCont)
    # print("NUMBER OF DISSOLVED COMMUNITIES: ", nDissolved)
    # print("NUMBER OF REAL COMMUNITIES: ", (communityCont-nDissolved))
    # print("NODES IN COMMUNITIES DAY FILE NO PRESENT IN DAY GRAPH: ", noMatchNodes)
    # print("NUMBER OF NODES IN COMMUNITIES DAY FILE NL PRESENT IN DAY GRAPH: ", len(noMatchNodes))
    # print("NODES IN DAY GRAPH NO PRESENT IN COMMUNITIES DAY FILE: ", nodesGraphNoMatch)
    # print("NUMBER OF NODES IN DAY GRAPH NO PRESENT IN COMMUNITIES DAY FILE: ", len(nodesGraphNoMatch))
    # print("NUMBER OF NODES IN COMMUNITIES DAY FILE PRESENT IN DAY GRAPH: ", nNodesPresent)
    # print("ALL DAY GRAPH AFTER REMOVING THE NO COMMUNITIES NODE: ", allCompactNetwork.summary())
    # print("----------------------------------")
    return communityAffiliates

def communityMemberTrend(communityAffiliates, communityId):
    
    daySituation = [0]*len(communityAffiliates)
    
    
    for day in communityAffiliates:
        if communityId in communityAffiliates[day]:
            if "DISSOLVED" not in communityAffiliates[day][communityId][0]:
                daySituation[day] = (len(communityAffiliates[day][communityId][0]), communityAffiliates[day][communityId][1], communityAffiliates[day][communityId][2])
            else: 
                daySituation[day] = ("DISSOLVED", 0)
        else:
            daySituation[day] = ("NOT PRESENT", 0)
    for i in range(0, len(daySituation)):
        print("{:<36}{:<3}".format("DAY: ", (i+1)))
        print("{:<36}{:<3}".format("NUMBER OF MEMBER ACTIVE IN THE NET: ", daySituation[i][0]))
        print("{:<36}{:<3}".format("TOTAL NUMBER IN COMMUNITY: ", daySituation[i][1]))
        print("{:<36}{:<3}".format("TOTAL NUMBER ACTIVE IN COMMUNITY: ", daySituation[i][2]))
        print("") 

### Communities number trend

In [10]:
def communitiesNumberTrend(communityAffiliates):
    
    with open(baseURL+'Dash/communitiesNumberTrend.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'type', 'quantity'])
    
        daySituation = [0]*len(communityAffiliates)
    
        for day in communityAffiliates:
            for community in communityAffiliates[day]:
                if "DISSOLVED" not in communityAffiliates[day][community][0]:
                    daySituation[day] += 1
    
        for i in range(0, len(daySituation)):
            print("{:<30}{:<3}".format("DAY: ", (i+1)))
            print("{:<30}{:<3}".format("NUMBER OF COMMUNITY IN THE NET: ", daySituation[i]))
            filewriter.writerow([i + 1, 'NumberOfCommunities', daySituation[i]])
        
            print("")           
    
def dissolvedCommunitiesTrend(communityAffiliates):
    
    with open(baseURL+'Dash/communitiesDissolvedTrend.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'type', 'quantity'])
    
        daySituation = [0]*len(communityAffiliates)
        nDissolved = 0
    
        for day in communityAffiliates:
            if day == 0:
                daySituation[day] = 0
                continue
            for community in communityAffiliates[day]:
                if "DISSOLVED" in communityAffiliates[day][community][0]:
                    daySituation[day] += 1
            if daySituation[day] == daySituation[day - 1]:
                daySituation[day] = 0
            else:
                daySituation[day] = daySituation[day] - nDissolved
                nDissolved += daySituation[day]
    
        for i in range(0, len(daySituation)):
            print("{:<42}{:<3}".format("DAY: ", (i+1)))
            print("{:<42}{:<3}".format("NUMBER OF DISSOLVED COMMUNITY IN THE NET: ", daySituation[i]))
            filewriter.writerow([i + 1, 'NumberOfDissolved', daySituation[i]])
            print("") 
        
    print(nDissolved)
        
def creationCommunitiesTrend(communityAffiliates):
    
    with open(baseURL+'Dash/communitiesCreationTrend.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'type', 'quantity'])
    
        daySituation = [0]*len(communityAffiliates)
    
        for day in communityAffiliates:
            if day == 0:
                daySituation[0] = 0
                continue
            if len(communityAffiliates[day]) - len(communityAffiliates[day - 1]) > 0:
                daySituation[day] = len(communityAffiliates[day]) - len(communityAffiliates[day - 1])
            else:
                daySituation[day] = 0
        
        for i in range(0, len(daySituation)):
            print("{:<42}{:<3}".format("DAY: ", (i+1)))
            print("{:<42}{:<3}".format("NUMBER OF COMMUNITIES CREATED IN THE NET: ", daySituation[i]))
            filewriter.writerow([i + 1, 'NumberOfCreated', daySituation[i]])
            print("") 

### Function for community/communities retriving

In [11]:
def retrieveCommunityGraph(allNetwork, communityId, netId):
    
    communityDayGraph = dict()
    dayGraph = ig.Graph(directed=True)
    nVertex = 0
    
    for day in range(0, 30):
        nodesWithNoInteraction = list()
        nVertex = 0
        dayGraph = ig.Graph(directed=True)
        for vertex in allNetwork[day][3].vs:
            if abs(vertex['Community']) == communityId:
                dayGraph.add_vertex(vertex['id'])
                dayGraph.vs[nVertex]['Community'] = vertex['Community'] 
                dayGraph.vs[nVertex]['label'] = vertex['id'] 
                nVertex += 1
                
        for edge in allNetwork[day][netId].es:
            for vertex in dayGraph.vs:
                if allNetwork[day][netId].vs[edge.tuple[0]]['id'] == vertex['name'] and abs(allNetwork[day][netId].vs[edge.tuple[1]]['Community']) == communityId:
                    toVertex = allNetwork[day][netId].vs[edge.tuple[1]]['id']
                    if  'Label' in edge.attribute_names():
                        if edge['Label'] == 'Attack':
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "red")
                        elif edge['Label'] == 'Message':
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "cornflowerblue")
                        else:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "orange")

                    elif 'MWeight' in edge.attribute_names() and 'TWeight' in edge.attribute_names():
                        if edge['MWeight'] > 0 and edge['TWeight'] > 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "darkgreen")
                        elif edge['MWeight'] > 0 and edge['TWeight'] == 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "cornflowerblue")
                        elif edge['MWeight'] == 0 and edge['TWeight'] > 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "orange")
                    else:
                        dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, Weight = edge['Weight'])
                        
        communityDayGraph[day] = dayGraph
    
    return communityDayGraph  


def retrieveGraph(allNetwork):
    
    communityDayMGraph = dict()
    communityDayTGraph = dict()
    
    communityDayMNoSameGraph = dict()
    communityDayTNoSameGraph = dict()
    
    communityDayMSameGraph = dict()
    communityDayTSameGraph = dict()
    
    dayMGraph = ig.Graph(directed=True)
    dayTGraph = ig.Graph(directed=True)
    
    dayMNoSameGraph = ig.Graph(directed=True)
    dayTNoSameGraph = ig.Graph(directed=True)
    
    dayMSameGraph = ig.Graph(directed=True)
    dayTSameGraph = ig.Graph(directed=True)
    nVertex = 0
    
    for day in tqdm(range(0, 30)):
        nVertex = 0
        dayMGraph = ig.Graph(directed=True)
        dayTGraph = ig.Graph(directed=True)
    
        dayMNoSameGraph = ig.Graph(directed=True)
        dayTNoSameGraph = ig.Graph(directed=True)
    
        dayMSameGraph = ig.Graph(directed=True)
        dayTSameGraph = ig.Graph(directed=True)
        
        
        for vertex in allNetwork[day][3].vs:
            dayMGraph.add_vertex(vertex['id'])
            dayMGraph.vs[nVertex]['label'] = vertex['id']
            dayMGraph.vs[nVertex]['Community'] = vertex['Community']
            
            dayTGraph.add_vertex(vertex['id'])
            dayTGraph.vs[nVertex]['label'] = vertex['id']
            dayTGraph.vs[nVertex]['Community'] = vertex['Community']
            
            dayMNoSameGraph.add_vertex(vertex['id'])
            dayMNoSameGraph.vs[nVertex]['label'] = vertex['id'] 
            dayMNoSameGraph.vs[nVertex]['Community'] = vertex['Community'] 
            
            dayTNoSameGraph.add_vertex(vertex['id'])
            dayTNoSameGraph.vs[nVertex]['label'] = vertex['id'] 
            dayTNoSameGraph.vs[nVertex]['Community'] = vertex['Community'] 
            
            dayMSameGraph.add_vertex(vertex['id'])
            dayMSameGraph.vs[nVertex]['label'] = vertex['id'] 
            dayMSameGraph.vs[nVertex]['Community'] = vertex['Community'] 
            
            dayTSameGraph.add_vertex(vertex['id'])
            dayTSameGraph.vs[nVertex]['label'] = vertex['id'] 
            dayTSameGraph.vs[nVertex]['Community'] = vertex['Community'] 
            
            nVertex += 1
                
        for edge in allNetwork[day][3].es:
            for vertex in dayMGraph.vs:
                if allNetwork[day][3].vs[edge.tuple[0]]['id'] == vertex['name']:
                    toVertex = allNetwork[day][3].vs[edge.tuple[1]]['id']
                    if edge['MWeight'] > 0:
                        if abs(allNetwork[day][3].vs[edge.tuple[1]]['Community']) != abs(allNetwork[day][3].vs[edge.tuple[0]]['Community']):
                            dayMNoSameGraph.add_edge(vertex.index, dayMGraph.vs.select(name = toVertex)[0].index, Weight = edge['MWeight'])
                        elif abs(allNetwork[day][3].vs[edge.tuple[1]]['Community']) == abs(allNetwork[day][3].vs[edge.tuple[0]]['Community']):
                            dayMSameGraph.add_edge(vertex.index, dayMGraph.vs.select(name = toVertex)[0].index, Weight = edge['MWeight'])
                        dayMGraph.add_edge(vertex.index, dayMGraph.vs.select(name = toVertex)[0].index, Weight = edge['MWeight'])
                        
                    if edge['TWeight'] > 0:
                        if abs(allNetwork[day][3].vs[edge.tuple[1]]['Community']) != abs(allNetwork[day][3].vs[edge.tuple[0]]['Community']):
                            dayTNoSameGraph.add_edge(vertex.index, dayMGraph.vs.select(name = toVertex)[0].index, Weight = edge['TWeight'])
                        elif abs(allNetwork[day][3].vs[edge.tuple[1]]['Community']) == abs(allNetwork[day][3].vs[edge.tuple[0]]['Community']):
                            dayTSameGraph.add_edge(vertex.index, dayMGraph.vs.select(name = toVertex)[0].index, Weight = edge['TWeight'])
                        dayTGraph.add_edge(vertex.index, dayMGraph.vs.select(name = toVertex)[0].index, Weight = edge['TWeight'])
        
        
        
        communityDayMGraph[day] = dayMGraph
        communityDayTGraph[day] = dayTGraph
    
        communityDayMNoSameGraph[day] = dayMNoSameGraph
        communityDayTNoSameGraph[day] = dayTNoSameGraph
    
        communityDayMSameGraph[day] = dayMSameGraph
        communityDayTSameGraph[day] = dayTSameGraph
        
    
    return communityDayMGraph, communityDayTGraph, communityDayMNoSameGraph, communityDayTNoSameGraph, communityDayMSameGraph, communityDayTSameGraph

def retrieveCommunitiesGraph(allNetwork, communityListId, netId):
    
    communityDayGraph = dict()
    dayGraph = ig.Graph(directed=True)
    nVertex = 0
    
    for day in range(0, 30):
        
        nodesWithNoInteraction = list()
        nVertex = 0
        dayGraph = ig.Graph(directed=True)
        for vertex in allNetwork[day][netId].vs:
            if abs(vertex['Community']) in communityListId:
                dayGraph.add_vertex(vertex['id'])
                dayGraph.vs[nVertex]['Community'] = vertex['Community'] 
                dayGraph.vs[nVertex]['label'] = vertex['id'] 
                if 'color' in vertex.attribute_names():
                    dayGraph.vs[nVertex]['color'] = vertex['color']
                nVertex += 1
                
        for edge in allNetwork[day][netId].es:
            for vertex in dayGraph.vs:
                if allNetwork[day][netId].vs[edge.tuple[0]]['id'] == vertex['name'] and (abs(allNetwork[day][netId].vs[edge.tuple[1]]['Community']) in communityListId or abs(allNetwork[day][netId].vs[edge.tuple[1]]['Community']) in communityListId):
                    toVertex = allNetwork[day][netId].vs[edge.tuple[1]]['id']
                    if  'Label' in edge.attribute_names():
                        if edge['Label'] == 'Attack':
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "red")
                        elif edge['Label'] == 'Message':
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "lightblue")
                        else:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "orange")

                    elif 'MWeight' in edge.attribute_names() and 'TWeight' in edge.attribute_names():
                        if edge['MWeight'] > 0 and edge['TWeight'] > 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "darkgreen")
                        elif edge['MWeight'] > 0 and edge['TWeight'] == 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "cornflowerblue")
                        elif edge['MWeight'] == 0 and edge['TWeight'] > 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "orange")
                    else:
                        dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, Weight = edge['Weight'])
        
        
        communityDayGraph[day] = dayGraph
    
    return communityDayGraph

def labeledGraph():

    for j in tqdm(range (0, 5)):
        for i in range(0, len(allNetwork)):
            # print(allNetwork[i][j].summary())
            communityAffiliation[i] = addCommunitiesLabel(allNetwork[i][j], i)
            # print(allNetwork[i][j].summary())
    #print("") 

# Study about major community in 30 days

## Diameter, in/out degree distribution, average degree and clustering coefficient for a community net

In [12]:
def startStaticAnalysis(allDayGraph):
    print("START STATIC ANALYSIS")
    
    allDayMGraph = retrieveCommunityGraph(allNetwork, allDayGraph[0].vs[0]['Community'], 1)
    allDayTGraph = retrieveCommunityGraph(allNetwork, allDayGraph[0].vs[0]['Community'], 2)
    
    print(len(allDayMGraph[0].vs()))
    print(len(allDayTGraph[0].vs()))
    print(len(allDayGraph[0].vs))
    
    computeDiameter(allDayGraph, "")
    computeDiameter(allDayMGraph, "M")
    computeDiameter(allDayTGraph, "T")
    print("")
    
    computeClusteringCoefficient(allDayGraph, "")
    computeClusteringCoefficient(allDayMGraph, "M")
    computeClusteringCoefficient(allDayTGraph, "T")
    print("")
        
    computeAveragePathLength(allDayGraph, "")
    computeAveragePathLength(allDayMGraph, "M")
    computeAveragePathLength(allDayTGraph, "T")
    print("")
    
    computeDensity(allDayGraph, "")
    computeDensity(allDayMGraph, "M")
    computeDensity(allDayTGraph, "T")
    print("")
    
    computeReciprocity(allDayGraph, "")
    computeReciprocity(allDayMGraph, "M")
    computeReciprocity(allDayTGraph, "T")
    print("")
    
    computeDegreeDistribution(allDayGraph)
    
def computeDiameter(allDayGraph, dayType):
    
    dayDiameter = list()
    
    for day in allDayGraph:
        dayDiameter.append(allDayGraph[day].diameter(directed=True, unconn=True))
        
    with open(baseURL+'Dash/SingleCommunityStudy/StaticAnalysis/D_diameter'+dayType+'Data.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'diameter'])
        
        for day in range(0, len(dayDiameter)):
            filewriter.writerow([day+1, dayDiameter[day]])
    print("DIAMETER ANALYSIS DONE")

def computeClusteringCoefficient(allDayGraph, dayType):
    
    dayClusteringCoefficient = list()

    for day in allDayGraph:
        dayClusteringCoefficient.append(allDayGraph[day].transitivity_undirected())
        
    with open(baseURL+'Dash/SingleCommunityStudy/StaticAnalysis/CC_clustering'+dayType+'CoefficientData.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'clusteringCoefficient'])
        
        for day in range(0, len(dayClusteringCoefficient)):
            filewriter.writerow([day+1, dayClusteringCoefficient[day]])
    
    print("CLUSTERING COEFFICIENT ANALYSIS DONE")

def computeAveragePathLength(allDayGraph, dayType):
    
    dayAveragePathLength = list()
    
    for day in allDayGraph:
        dayAveragePathLength.append(allDayGraph[day].average_path_length(directed = True, unconn = True))
        
    with open(baseURL+'Dash/SingleCommunityStudy/StaticAnalysis/AP_average'+dayType+'PathLengthData.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'averagePathLength'])
        
        for day in range(0, len(dayAveragePathLength)):
            filewriter.writerow([day+1, dayAveragePathLength[day]])
    
    print("AVERAGE PATH LENGTH ANALYSIS DONE")
    

def computeDensity(allDayGraph, dayType):
    
    dayDensity = list()
    
    for day in allDayGraph:
        dayDensity.append(allDayGraph[day].density(loops = False))
        
    with open(baseURL+'Dash/SingleCommunityStudy/StaticAnalysis/DE_density'+dayType+'Data.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'density'])
        
        for day in range(0, len(dayDensity)):
            filewriter.writerow([day + 1, dayDensity[day]])
    
    print("DENSITY ANALYSIS DONE")

def computeReciprocity(allDayGraph, dayType):
    
    dayReciprocity = list()
    
     #reciprocity(ignore_loops=True, mode="default")
    
    for day in allDayGraph:
        dayReciprocity.append(allDayGraph[day].reciprocity(ignore_loops=True, mode=1))
        
    with open(baseURL+'Dash/SingleCommunityStudy/StaticAnalysis/RE_reciprocity'+dayType+'Data.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'Diameter'])
        
        for day in range(0, len(dayReciprocity)):
            filewriter.writerow([day+1, dayReciprocity[day]])
    print("RECIPROCITY ANALYSIS DONE")  
    
def computeDegreeDistribution(allDayGraph):
    
    if 'MWeight' in allDayGraph[0].es[0].attribute_names():
        allianceInDegreeDistribution(allDayGraph)
        allianceOutDegreeDistribution(allDayGraph)
        
def allianceInDegreeDistribution(allDayGraph):
    
    dayMessageDistribution = dict()
    dayTradeDistribution = dict()
    dayTotalDistribution = dict()
    
    alreadyManage = list()
    
    nZeroIndegree = 0
    nZeroInMessage = 0
    nZeroInTrade = 0
    
    nMessageIn = 0
    nTradeIn = 0
    nTotalIn = 0
    
    for day in allDayGraph:
        print("DAY: ", day + 1)
        print(allDayGraph[day].summary())
        dayMessageDistribution[0] = 0
        dayTradeDistribution[0] = 0
        dayTotalDistribution[0] = 0
        
        
        for vertex in allDayGraph[day].vs:
            if vertex.indegree() == 0 and vertex.outdegree() == 0:
                nZeroIndegree += 1
                continue 
            for edge in allDayGraph[day].es:
                if allDayGraph[day].vs[edge.tuple[1]]['name'] == vertex['name']:
                    nMessageIn += edge['MWeight']
                    nTradeIn += edge['TWeight']

            if nMessageIn in dayMessageDistribution:
                dayMessageDistribution[nMessageIn] += 1
            else:
                dayMessageDistribution[nMessageIn] = 1
            
            if nTradeIn in dayTradeDistribution:
                dayTradeDistribution[nTradeIn] += 1
            else:
                dayTradeDistribution[nTradeIn] = 1
                
            if (nTradeIn+nMessageIn) in dayTotalDistribution:
                dayTotalDistribution[(nTradeIn + nMessageIn)] += 1
            else:
                dayTotalDistribution[(nTradeIn + nMessageIn)] = 1
                
            nMessageIn = 0
            nTradeIn = 0
        
        dayMessageDistribution[0] += nZeroIndegree
        dayTradeDistribution[0] += nZeroIndegree
        dayTotalDistribution[0] += nZeroIndegree
                    
        print("MESSAGE: ", dayMessageDistribution)
        print("TRADE: ", dayTradeDistribution)
        print("TOTAL: ", dayTotalDistribution)
    
        nNodesCheck = 0
        for key in dayMessageDistribution:
            nNodesCheck += dayMessageDistribution[key]
        print("NODES CHECK ON DAY MESSAGE DISTRIBUTION: ", nNodesCheck)

        nNodesCheck = 0
        for key in dayTradeDistribution:
            nNodesCheck += dayTradeDistribution[key] 
        print("NODES CHECK ON DAY TRADE DISTRIBUTION: ", nNodesCheck)
    
        nNodesCheck = 0
        for key in dayTotalDistribution:
            nNodesCheck += dayTotalDistribution[key]  
        print("NODES CHECK ON DAY TOTAL DISTRIBUTION: ", nNodesCheck)
        print("")
        
        if day == 0:
            with open(baseURL+"Dash/SingleCommunityStudy/StaticAnalysis/indegreeDistribution.csv", 'a+') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
                
                filewriter.writerow(['day', 'type', 'range', 'value'])
            
                
                
        writeDataInFile(dayMessageDistribution, 'Message', (len(allDayGraph[day].vs)), day, 'in')
        writeDataInFile(dayTradeDistribution, 'Trade', (len(allDayGraph[day].vs)), day, 'in')
        writeDataInFile(dayTotalDistribution, 'Total', (len(allDayGraph[day].vs)), day, 'in')
            
        
        dayMessageDistribution = dict()
        dayTradeDistribution = dict()
        dayTotalDistribution = dict()
        nZeroIndegree = 0
                    
                
def writeDataInFile(dayDistribution, typeOfDistribution, nNodes, day, typeEdge):
    
        finalDayDistribution = dict()
    
        finalDayDistribution[0] = 0
        finalDayDistribution['1-3'] = 0
        finalDayDistribution['4-6'] = 0
        finalDayDistribution['7-9'] = 0
        finalDayDistribution['>9'] = 0
        
        startBound = 1
        endBound = 3
        nEdges = 0
    
        finalDayDistribution[0] = dayDistribution[0]
    
        while endBound <= 9:
            for key in dayDistribution:
                nEdges += key * dayDistribution[key]
                if key > 9:
                    finalDayDistribution[">9"] += dayDistribution[key]
                elif key >= startBound and key <= endBound:
                    finalDayDistribution[""+str(startBound)+"-"+str(endBound)+""] += dayDistribution[key]
            endBound += 3
            startBound += 3
        
        finalDayDistribution[">9"] = finalDayDistribution[">9"]/3
        nEdges = nEdges/3
        nodesCont = 0
        totalDistribution = 0
        
        for key in finalDayDistribution:
            nodesCont += int(finalDayDistribution[key])
            print(key, finalDayDistribution[key])
            totalDistribution += finalDayDistribution[key]/nNodes
            
    
        print(nodesCont)
        print(totalDistribution)
        print("")
        
        with open(baseURL+"Dash/SingleCommunityStudy/StaticAnalysis/"+typeEdge+"degreeDistribution.csv", 'a+') as csvfile:
            filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)

            for key in finalDayDistribution:
                value = round(finalDayDistribution[key]/nNodes, 9)
                
                if day == 9:
                    print("HERE MAN: ", finalDayDistribution)
                    print("DENOMINATOR: ", nNodes)
                    print("VALUE: ", value)
                    value = round(finalDayDistribution[key]/(nNodes + 0.0001), 9)
                filewriter.writerow([day + 1, typeOfDistribution, key, value])
        
        if typeEdge == 'in':
            with open(baseURL+"Dash/SingleCommunityStudy/StaticAnalysis/average"+str(typeOfDistribution)+"Degree.csv", 'a+') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            
                print(typeOfDistribution, ": ", nEdges)
                if day == 0:
                    filewriter.writerow(["day", "averageDegree"])
        
                filewriter.writerow([day + 1, round(nEdges/nNodes+(0.00001), 9)])
    
def allianceOutDegreeDistribution(allDayGraph):
    
    dayMessageDistribution = dict()
    dayTradeDistribution = dict()
    dayTotalDistribution = dict()
    
    finalDayDistribution = dict()
    
    
    nZeroOutdegree = 0
    
    nMessageOut = 0
    nTradeOut = 0
    nTotalOut = 0
    
    for day in allDayGraph:
        print("DAY: ", day + 1)
        print(allDayGraph[day].summary())
        dayMessageDistribution[0] = 0
        dayTradeDistribution[0] = 0
        dayTotalDistribution[0] = 0
        
        
        for vertex in allDayGraph[day].vs:
            if vertex.indegree() == 0 and vertex.outdegree() == 0:
                nZeroOutdegree += 1
                continue 
            for edge in allDayGraph[day].es:
                if allDayGraph[day].vs[edge.tuple[0]]['name'] == vertex['name']:
                    nMessageOut += edge['MWeight']
                    nTradeOut += edge['TWeight']

        
            if nMessageOut in dayMessageDistribution:
                dayMessageDistribution[nMessageOut] += 1
            else:
                dayMessageDistribution[nMessageOut] = 1
            
            if nTradeOut in dayTradeDistribution:
                dayTradeDistribution[nTradeOut] += 1
            else:
                dayTradeDistribution[nTradeOut] = 1
                
            if (nTradeOut+nMessageOut) in dayTotalDistribution:
                dayTotalDistribution[(nTradeOut + nMessageOut)] += 1
            else:
                dayTotalDistribution[(nTradeOut + nMessageOut)] = 1
                
            nMessageOut = 0
            nTradeOut = 0
        
        dayMessageDistribution[0] += nZeroOutdegree
        dayTradeDistribution[0] += nZeroOutdegree
        dayTotalDistribution[0] += nZeroOutdegree
                    
        print("MESSAGE: ", dayMessageDistribution)
        print("TRADE: ", dayTradeDistribution)
        print("TOTAL: ", dayTotalDistribution)
    
        nNodesCheck = 0
        for key in dayMessageDistribution:
            nNodesCheck += dayMessageDistribution[key]
        print("NODES CHECK ON DAY MESSAGE DISTRIBUTION: ", nNodesCheck)

        nNodesCheck = 0
        for key in dayTradeDistribution:
            nNodesCheck += dayTradeDistribution[key] 
        print("NODES CHECK ON DAY TRADE DISTRIBUTION: ", nNodesCheck)
    
        nNodesCheck = 0
        for key in dayTotalDistribution:
            nNodesCheck += dayTotalDistribution[key]  
        print("NODES CHECK ON DAY TOTAL DISTRIBUTION: ", nNodesCheck)
        print("")
        
        if day == 0:
            with open(baseURL+"Dash/SingleCommunityStudy/StaticAnalysis/outdegreeDistribution.csv", 'a+') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
                
                filewriter.writerow(['day', 'type', 'range', 'value'])
            
                
                
        writeDataInFile(dayMessageDistribution, 'Message', (len(allDayGraph[day].vs)), day, 'out')
        writeDataInFile(dayTradeDistribution, 'Trade', (len(allDayGraph[day].vs)), day, 'out')
        writeDataInFile(dayTotalDistribution, 'Total', (len(allDayGraph[day].vs)), day, 'out')
            
        
        dayMessageDistribution = dict()
        dayTradeDistribution = dict()
        dayTotalDistribution = dict()
        nZeroOutdegree = 0

### Diplomats Analysis

In [29]:
def findDiplomats(allGraph, communityId):
    print("DIPLOMATS DETECTION")
    
    diplomats = dict()
    orderedDiplomats = list()
    
    for day in allGraph:
        nToPrint = 5
        print("DAY: ", (day+1))
        edgeBet = allGraph[day][1].edge_betweenness()
        orderEdgeBet = np.argsort(edgeBet)[::-1]
        orderEdgeBet = np.argsort(edgeBet)[::-1]
        print("{:>5}  {:>5}  {:>14}  {:>14}  {:>16}  {:>13}".
          format("From", "To", "From Community", "To Community", "Edge Betweennees", "Weight"))
        for e in orderEdgeBet:
            if allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['Community'] != allGraph[day][1].vs[allGraph[day][1].es[e].tuple[1]]['Community'] and abs(allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['Community']) == communityId:
                if nToPrint == 0:
                    break
                print("{:>5}  {:>5}  {:>14}  {:>14}  {:>16}  {:>13}".
                    format(allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['id'], allGraph[day][1].vs[allGraph[day][1].es[e].tuple[1]]['id'], 
                             allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['Community'], allGraph[day][1].vs[allGraph[day][1].es[e].tuple[1]]['Community'], 
                             round(edgeBet[e], 5), round(allGraph[day][1].es[e]['Weight']), 0))
                if allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['id'] in diplomats:
                    diplomats[allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['id']] += 1
                else:
                    diplomats[allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['id']] = 1
                nToPrint -=1
    
    orderedDiplomats = sorted(diplomats, key=diplomats.get, reverse = True)
    
    if len(orderedDiplomats) > 4:
        orderedDiplomats = orderedDiplomats[0:4]
        
    for day in allGraph:
        for vertex in communityGraph[day].vs:
            if vertex['label'] in orderedDiplomats:
                vertex['color'] = 'cornflowerblue'
    print("")
    print(orderedDiplomats)
    
    
def averageRelationship(communityGraph):
    nRed = 0
    nBlue = 0
    
    nMessage = dict()
    
    averageBlue = list()
    averageRed = list()
    
    for day in communityGraph:
        nMessage["blue"] = 0
        nMessage["red"] = 0
        
        nRed = 0
        nBlue = 0

        for edge in communityGraph[day].es:
            if communityGraph[day].vs[edge.tuple[0]]['color'] is None and communityGraph[day].vs[edge.tuple[1]]['color'] == 'yellow':
                nMessage["red"] += edge['MWeight']
                nRed += 1
            if communityGraph[day].vs[edge.tuple[1]]['color'] is None and communityGraph[day].vs[edge.tuple[0]]['color'] == 'yellow':
                nMessage["red"] += edge['MWeight']
                nRed += 1
            if communityGraph[day].vs[edge.tuple[0]]['color'] == 'cornflowerblue' and communityGraph[day].vs[edge.tuple[1]]['color'] == 'yellow':
                nMessage["blue"] += edge['MWeight']
                nBlue += 1
            if communityGraph[day].vs[edge.tuple[1]]['color'] == 'cornflowerblue' and communityGraph[day].vs[edge.tuple[0]]['color'] == 'yellow':
                nMessage["blue"] += edge['MWeight']
                nBlue += 1
            
        if nBlue > 0:
            averageBlue.append(nMessage["blue"]/nBlue)
        else:
            averageBlue.append(0)
            
        if nRed > 0:
            averageRed.append(nMessage["red"]/nRed)
        else:
            averageRed.append(0)
    
    print("AVERAGE BLUE MESSAGE WITH LEADER: ", sum(averageBlue)/len(averageBlue))
    print("AVERAGE RED MESSAGE WITH LEADER: ", sum(averageRed)/len(averageRed))
    
    with open(baseURL+'Dash/SingleCommunityStudy/DiplomatsVsNoDiplomats/data.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['NormalUserLeaderMessage', 'DiplomatsLeaderMessage'])
        filewriter.writerow([sum(averageRed)/len(averageRed), sum(averageBlue)/len(averageBlue)])

## Community leader search
#### Community Centrality
In this cell we compute all main centrality measurement for finding a top-two central node. All based on a specific community graphs on the period (from day 1 to 30 day).

In [49]:
def computeCentralityMeasurement(communityStory):
    
    messageGraphCommunity = dict()
    tradeGraphCommunity = dict()
    
    messageGraphCommunity = retrieveCommunityGraph(allNetwork, communityStory[0].vs[0]['Community'], 1)
    tradeGraphCommunity = retrieveCommunityGraph(allNetwork, communityStory[0].vs[0]['Community'], 2)
    
    for day in communityStory:
        if len(communityStory[day].vs()) == len(messageGraphCommunity[day].vs()) and len(messageGraphCommunity[day].vs()) == len(tradeGraphCommunity[day].vs()):
            continue
        else:
            print("PROBLEM TO DAY: ", day + 1 )
            
    if os.path.exists(baseURL+'Dash/SingleCommunityStudy/CSVFILE/degreeTopResult.csv'):
          os.remove(baseURL+'Dash/SingleCommunityStudy/CSVFILE/degreeTopResult.csv')
            
    if os.path.exists(baseURL+'Dash/SingleCommunityStudy/CSVFILE/betweennesTopResult.csv'):
          os.remove(baseURL+'Dash/SingleCommunityStudy/CSVFILE/betweennesTopResult.csv')
            
    if os.path.exists(baseURL+'Dash/SingleCommunityStudy/CSVFILE/closenessTopResult.csv'):
          os.remove(baseURL+'Dash/SingleCommunityStudy/CSVFILE/closenessTopResult.csv')
            
    if os.path.exists(baseURL+'Dash/SingleCommunityStudy/CSVFILE/eigenvectorTopResult.csv'):
          os.remove(baseURL+'Dash/SingleCommunityStudy/CSVFILE/eigenvectorTopResult.csv')
     
    computeBetweenness(communityStory, messageGraphCommunity, tradeGraphCommunity)
    computeCloseness(communityStory, messageGraphCommunity, tradeGraphCommunity)
    computeEigenvector(communityStory)
    computeDegree(communityStory, messageGraphCommunity, tradeGraphCommunity)
    
    for day in communityGraph:
        for vertex in communityGraph[day].vs:
            if vertex['label'] == list(top5All)[0] or vertex['label'] == list(top5All)[1]:
                vertex['color'] = 'yellow'

    
def computeDegree(communityStory, messageGraphCommunity, tradeGraphCommunity):
    
    daysSummaryMessage = dict()
    daysSummaryTrade = dict()
    daysSummaryTotal = dict()
    
    nMessageOut = 0
    nTradeOut = 0
    nTotalOut = 0
    
    for day in communityStory:
        
        daysSummaryMessage = dict()
        daysSummaryTrade = dict()
        daysSummaryTotal = dict()
        
        
        for vertex in communityStory[day].vs:
            
            nMessageOut = 0
            nTradeOut = 0
            nTotalOut = 0
            
            for edge in communityStory[day].es:
                if communityStory[day].vs[edge.tuple[0]]['label'] == vertex['label']:
                    if edge['MWeight'] > 0:
                        nMessageOut += 1
                    if edge['TWeight'] > 0:
                        nTradeOut += 1
                    nTotalOut += 1
                    
            communityStory[day].vs.select(label = vertex['label'])['MDegree'] = nMessageOut/(len(communityStory[day].vs) - 1)
            daysSummaryMessage[vertex['label']] = nMessageOut/(len(communityStory[day].vs) - 1)
            
            communityStory[day].vs.select(label = vertex['label'])['TDegree'] = nTradeOut/(len(communityStory[day].vs) - 1)
            daysSummaryTrade[vertex['label']] = nTradeOut/(len(communityStory[day].vs) - 1)
            
            communityStory[day].vs.select(label = vertex['label'])['Degree'] = nTotalOut/(len(communityStory[day].vs) - 1)
            daysSummaryTotal[vertex['label']] = nTotalOut/(len(communityStory[day].vs) - 1)
            
        with open(baseURL+'Dash/SingleCommunityStudy/CSVFILE/degreeTopResult.csv', 'a') as csvfile:
            filewriter = csv.writer(csvfile, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            if day == 0:
                filewriter.writerow(['day', 'type', 'player', 'value'])
        
            for player, value in sorted(daysSummaryMessage.items(), key=lambda item: item[1], reverse = True)[0:5]:
                filewriter.writerow([day + 1, 'Message', player, value])
            
            for player, value in sorted(daysSummaryTrade.items(), key=lambda item: item[1], reverse = True)[0:5]:
                filewriter.writerow([day + 1, 'Trade', player, value])
                
            position = 0
            for player, value in sorted(daysSummaryTotal.items(), key=lambda item: item[1], reverse = True)[0:5]:
                if position == 0:
                    if player in top5All:
                        top5All[player] += 1
                    else:
                        top5All[player] = 1
                filewriter.writerow([day + 1, 'Total', player, value])
                position += 1

def computeBetweenness(communityStory, messageGraphCommunity, tradeGraphCommunity):
    
    daysSummaryMessage = dict()
    daysSummaryTrade = dict()
    daysSummaryTotal = dict()
    
    vertexMBetweenness = 0
    vertexTBetweenness = 0
    vertexBetweenness = 0
    
    for day in communityStory:
        daysSummaryMessage = dict()
        daysSummaryTrade = dict()
        daysSummaryTotal = dict()
        
        for vertex in communityStory[day].vs:
            
            vertexMBetweenness = messageGraphCommunity[day].vs.select(label = vertex['label']).betweenness(directed=True)
            if vertexMBetweenness == []:
                vertex['MBetweenness'] = 0.0
            else:
                vertexMBetweenness = vertexMBetweenness[0]/((len(communityStory[day].vs)-1)*(len(communityStory[day].vs)-2))
                vertex['MBetweenness'] = vertexMBetweenness
            daysSummaryMessage[vertex['label']] = vertex['MBetweenness']
        
            vertexTBetweenness = tradeGraphCommunity[day].vs.select(label = vertex['label']).betweenness(directed=True)
            if vertexTBetweenness == []:
                vertex['TBetweenness'] = 0.0
            else:
                vertexTBetweenness = vertexTBetweenness[0]/((len(communityStory[day].vs)-1)*(len(communityStory[day].vs)-2))
                vertex['TBetweenness'] = vertexTBetweenness
            daysSummaryTrade[vertex['label']] = vertex['TBetweenness']
            
            vertexBetweenness = communityStory[day].vs.select(label = vertex['label']).betweenness(directed=True)
            if vertexBetweenness == []:
                vertex['Betweenness'] = 0.0
            else:
                vertexBetweenness = vertexBetweenness[0]/((len(communityStory[day].vs)-1)*(len(communityStory[day].vs)-2))
                vertex['Betweenness'] = vertexBetweenness
            daysSummaryTotal[vertex['label']] = vertex['Betweenness']
        
        with open(baseURL+'Dash/SingleCommunityStudy/CSVFILE/betweennesTopResult.csv', 'a') as csvfile:
            filewriter = csv.writer(csvfile, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            if day == 0:
                filewriter.writerow(['day', 'type', 'player', 'value'])
        
            for player, value in sorted(daysSummaryMessage.items(), key=lambda item: item[1], reverse = True)[0:5]:
                filewriter.writerow([day + 1, 'Message', player, value])
            
            for player, value in sorted(daysSummaryTrade.items(), key=lambda item: item[1], reverse = True)[0:5]:
                filewriter.writerow([day + 1, 'Trade', player, value])
            
            position = 0
            for player, value in sorted(daysSummaryTotal.items(), key=lambda item: item[1], reverse = True)[0:5]:
                if position == 0:
                    if player in top5All:
                        top5All[player] += 1
                    else:
                        top5All[player] = 1
                filewriter.writerow([day + 1, 'Total', player, value])
                position += 1
                
        
def computeCloseness(communityStory, messageGraphCommunity, tradeGraphCommunity):
    
    daysSummaryMessage = dict()
    daysSummaryTrade = dict()
    daysSummaryTotal = dict()
    
    vertexMCloseness = 0
    vertexTCloseness = 0
    vertexCloseness = 0
    
    vertexMOut = False
    
    for day in communityStory:
        daysSummaryMessage = dict()
        daysSummaryTrade = dict()
        daysSummaryTotal = dict()
        vertexMOut = False
        
        for vertex in communityStory[day].vs:
            
            vertexMCloseness = messageGraphCommunity[day].vs.select(label = vertex['label']).closeness(normalized=True, mode = 1)
            if vertexMCloseness == []:
                vertex['MCloseness'] = 0.0
                vertexMOut = True
            else:
                vertex['MCloseness'] = vertexMCloseness[0]
            daysSummaryMessage[vertex['label']] = vertex['MCloseness']
        
            vertexTCloseness = tradeGraphCommunity[day].vs.select(label = vertex['label']).closeness(normalized=True, mode = 1)
            if vertexTCloseness == []:
                vertex['TCloseness'] = 0.0
            else:
                vertex['TCloseness'] = vertexTCloseness[0]
            daysSummaryTrade[vertex['label']] = vertex['TCloseness']
            
            vertexCloseness = communityStory[day].vs.select(label = vertex['label']).closeness(normalized=True, mode = 1)
            if vertexCloseness == []:
                vertex['Closeness'] = 0.0
            else:
                vertex['Closeness'] = vertexCloseness[0]
            daysSummaryTotal[vertex['label']] = vertex['Closeness']
        
        with open(baseURL+'Dash/SingleCommunityStudy/CSVFILE/closenessTopResult.csv', 'a') as csvfile:
            filewriter = csv.writer(csvfile, delimiter=';',quotechar='/', quoting=csv.QUOTE_MINIMAL)
            
            if day == 0:
                filewriter.writerow(['day', 'type', 'player', 'value'])
            
            if day == 10:
                print(len(messageGraphCommunity[day].components(mode=1)))
                print(messageGraphCommunity[day].components(mode=1))
            
            if len(messageGraphCommunity[day].components(mode=1)) == 1 and not vertexMOut:
                for player, value in sorted(daysSummaryMessage.items(), key=lambda item: item[1], reverse = True)[0:5]:
                    filewriter.writerow([day + 1, 'Message', player, value])
            
            if len(tradeGraphCommunity[day].components(mode=1)) == 1:
                for player, value in sorted(daysSummaryTrade.items(), key=lambda item: item[1], reverse = True)[0:5]:
                    filewriter.writerow([day + 1, 'Trade', player, value])
            
            position = 0
            if len(communityStory[day].components(mode=1)) == 1:
                for player, value in sorted(daysSummaryTotal.items(), key=lambda item: item[1], reverse = True)[0:5]:
                    if position == 0:
                        if player in top5All:
                            top5All[player] += 1
                        else:
                            top5All[player] = 1
                    filewriter.writerow([day + 1, 'Total', player, value])
                    position += 1
    
    
def computeEigenvector(communityStory): #, messageGraphCommunity, tradeGraphCommunity):
    
    daysSummaryTotal = dict()
    
    for day in communityStory:
        daysSummaryTotal = dict()
        print("DAY: ", day+1)
        
        communityStory[day].vs['Eigenvector'] = communityStory[day].eigenvector_centrality(directed=True)
        
        
        for vertex in communityStory[day].vs:
            daysSummaryTotal[vertex['label']] = vertex['Eigenvector']
        
        with open(baseURL+'Dash/SingleCommunityStudy/CSVFILE/eigenvectorTopResult.csv', 'a') as csvfile:
            filewriter = csv.writer(csvfile, delimiter=';',quotechar='/', quoting=csv.QUOTE_MINIMAL)
            
            if day == 0:
                filewriter.writerow(['day', 'type', 'player', 'value'])
            
            position = 0
            for player, value in sorted(daysSummaryTotal.items(), key=lambda item: item[1], reverse = True)[0:5]:
                if position == 0:
                    if player in top5All:
                        top5All[player] += 1
                    else:
                        top5All[player] = 1
                filewriter.writerow([day + 1, 'Total', player, value])
                position += 1

def deleteLeader(communityGraph):
    
    deletedGraph = communityGraph
    nodeToDelete = list()
    
    for day in deletedGraph:
        nodeToDelete = list()
        for vertex in deletedGraph[day].vs:
            if vertex['color'] == 'yellow':
                nodeToDelete.append(vertex.index)
        deletedGraph[day].delete_vertices(nodeToDelete)
    
        plotToSave = ig.plot(deletedGraph[day])
        plotToSave.save(baseURL +'Dash/SingleCommunityStudy/LeaderDelete/day'+str(day+1)+'.png') 

## Saved community plot

In [53]:
def saveAllDaysPlot(communityGraph):
    
    for day in communityGraph:
        plotToSave = ig.plot(communityGraph[day])
        plotToSave.save(baseURL +'Dash/SingleCommunityStudy/PlotAllDays/day'+str(day+1)+'.png')
        
def saveAllDaysBetPlot(communityGraph):
    # ig.plot(communityGraph[8], vertex_size = 100*np.array(communityGraph[8].vs['Betweenness']) + 9)
    for day in communityGraph:
        plotMBetweenness = ig.plot(communityGraph[day], vertex_size = 200*np.array(communityGraph[day].vs['MBetweenness']) + 9)
        plotTBetweenness = ig.plot(communityGraph[day], vertex_size = 10000*np.array(communityGraph[day].vs['TBetweenness']) + 9)
        plotBetweenness = ig.plot(communityGraph[day], vertex_size = 200*np.array(communityGraph[day].vs['Betweenness']) + 9)
        plotMBetweenness.save(baseURL +'Dash/SingleCommunityStudy/BE_PlotMBet/day'+str(day+1)+'.png')
        plotTBetweenness.save(baseURL +'Dash/SingleCommunityStudy/BE_PlotTBet/day'+str(day+1)+'.png')
        plotBetweenness.save(baseURL +'Dash/SingleCommunityStudy/BE_PlotBet/day'+str(day+1)+'.png')
        
def saveAllDaysClosePlot(communityGraph):
    for day in tqdm(range(0, len(communityGraph))):
        if day == 11:
            continue
        plotMCloseness = ig.plot(communityGraph[day], vertex_size = 40*np.array(communityGraph[day].vs['MCloseness']) + 9)
        plotTCloseness = ig.plot(communityGraph[day], vertex_size = 40*np.array(communityGraph[day].vs['TCloseness']) + 9)
        plotCloseness = ig.plot(communityGraph[day], vertex_size = 40*np.array(communityGraph[day].vs['Closeness']) + 9)
        plotMCloseness.save(baseURL +'Dash/SingleCommunityStudy/C_PlotMClose/day'+str(day+1)+'.png')
        plotTCloseness.save(baseURL +'Dash/SingleCommunityStudy/C_PlotTClose/day'+str(day+1)+'.png')
        plotCloseness.save(baseURL +'Dash/SingleCommunityStudy/C_PlotClose/day'+str(day+1)+'.png')
        
def saveAllDaysEigenPlot(communityGraph):
    for day in communityGraph:
        plotEigen = ig.plot(communityGraph[day], vertex_size = 20*np.array(communityGraph[day].vs['Eigenvector']) + 9)
        plotEigen.save(baseURL +'Dash/SingleCommunityStudy/E_PlotEigen/day'+str(day+1)+'.png')
        
def saveAllDaysDegreePlot(communityGraph):
    for day in tqdm(range(0, len(communityGraph))):
        if day == 11:
            continue
        else:
            plotMDegree = ig.plot(communityGraph[day], vertex_size = 40*np.array(communityGraph[day].vs['MDegree']) + 9)
            plotTDegree = ig.plot(communityGraph[day], vertex_size = 200*np.array(communityGraph[day].vs['TDegree']) + 9)
            plotDegree = ig.plot(communityGraph[day], vertex_size = 40*np.array(communityGraph[day].vs['Degree']) + 9)
            
        plotMDegree.save(baseURL +'Dash/SingleCommunityStudy/D_PlotMDegree/day'+str(day+1)+'.png')
        plotTDegree.save(baseURL +'Dash/SingleCommunityStudy/D_PlotTDegree/day'+str(day+1)+'.png')
        plotDegree.save(baseURL +'Dash/SingleCommunityStudy/D_PlotDegree/day'+str(day+1)+'.png')

# Compute average about all measure done in the period

In [16]:
def computeAllAverage():
    
    averageTotalDegree()
    averageTotalShortestPath()
    

def averageTotalDegree():
    
    dayTrendAttack = list()
    dayTrendTrade = list()
    dayTrendMessage = list()
    
    with open(baseURL+"Dash/AverageDegree/AttackGraphData.csv") as csvfile:
        filereader = csv.reader(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        for line in filereader:
            if line[1] == 'AverageDegree':
                continue
            dayTrendAttack.append(float(line[1]))
            
    with open(baseURL+"Dash/AverageDegree/TradeGraphData.csv") as csvfile:
        filereader = csv.reader(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        for line in filereader:
            if line[1] == 'AverageDegree':
                continue
            dayTrendTrade.append(float(line[1]))
            
    with open(baseURL+"Dash/AverageDegree/MessageGraphData.csv") as csvfile:
        filereader = csv.reader(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        for line in filereader:
            if line[1] == 'AverageDegree':
                continue
            dayTrendMessage.append(float(line[1]))
        
    with open(baseURL+"Dash/AverageDegree/AverageData.csv", "w") as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            
        filewriter.writerow(['Type', 'AverageValue'])
        
        filewriter.writerow(['Attack', round(sum(dayTrendAttack)/len(dayTrendAttack), 2)])
        filewriter.writerow(['Trade', round(sum(dayTrendTrade)/len(dayTrendTrade), 2)])
        filewriter.writerow(['Message', round(sum(dayTrendMessage)/len(dayTrendMessage), 2)])

def averageTotalShortestPath():
    
    dayTrendAttack = list()
    dayTrendTrade = list()
    dayTrendMessage = list()
    
    with open(baseURL+"Dash/AverageShortestPath/AttackGraphData.csv") as csvfile:
        filereader = csv.reader(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        for line in filereader:
            if line[1] == 'AverageShortestPath':
                continue
            dayTrendAttack.append(float(line[1]))
            
    with open(baseURL+"Dash/AverageShortestPath/TradeGraphData.csv") as csvfile:
        filereader = csv.reader(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        for line in filereader:
            if line[1] == 'AverageShortestPath':
                continue
            dayTrendTrade.append(float(line[1]))
            
    with open(baseURL+"Dash/AverageShortestPath/MessageGraphData.csv") as csvfile:
        filereader = csv.reader(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        for line in filereader:
            if line[1] == 'AverageShortestPath':
                continue
            dayTrendMessage.append(float(line[1]))
        
    with open(baseURL+"Dash/AverageShortestPath/AverageData.csv", "w") as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            
        filewriter.writerow(['Type', 'AverageValue'])
        
        filewriter.writerow(['Attack', round(sum(dayTrendAttack)/len(dayTrendAttack), 2)])
        filewriter.writerow(['Trade', round(sum(dayTrendTrade)/len(dayTrendTrade), 2)])
        filewriter.writerow(['Message', round(sum(dayTrendMessage)/len(dayTrendMessage), 2)])
        

    
    

## Main
From cell below we let's start all our analysis

In [54]:
#userNumberTrend(allNetwork)                           
#edgeNumberTrend(allNetwork)  

#tricksterAnalysis(allNetwork)

#communityAffiliation = dict()
#labeledGraph()

#### COMPUTE CLUSTERING, DENSITY, RECIPROCITY, DIAMETER, AVERAGEPATH, AVERAGEDEGREE, IN/OUT DEGREE DISTRIBUTION
# computeStaticOnTotal(allNetwork)

# communitiesNumberTrend(communityAffiliation)          
# dissolvedCommunitiesTrend(communityAffiliation)       
# creationCommunitiesTrend(communityAffiliation)

# top5All = dict()
# communityGraph = retrieveCommunitiesGraph(allNetwork, (5, 5), 3) 
# startStaticAnalysis(communityGraph)
# communityMemberTrend(communityAffiliation, 5)
# findDiplomats(allNetwork, 5)
# print(communityGraph[0].vs['Eigenvector'])
# computeCentralityMeasurement(communityGraph)
# averageRelationship(communityGraph) 
# deleteLeader(communityGraph)


#saveAllDaysPlot(communityGraph)                       
#saveAllDaysBetPlot(communityGraph)                    
#saveAllDaysClosePlot(communityGraph)                  
#saveAllDaysEigenPlot(communityGraph)                  
#saveAllDaysDegreePlot(communityGraph) 


#computeAllAverage()

# Creation of compact graph (attack, trade and message) and Total Graph and the Alliance Graph (compact and not)
You don't have to run this code if you already have all the graph files in your dataset graphml folder; but **run for another one check**


## Create compact attack graph

In [4]:
# Creation of 30 plain graphs
nGraph = 30
compactAttackGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

attackCount = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    attackGraph = nx.MultiDiGraph()
    compactAttackGraph = nx.DiGraph()
    checkGraph = ig.Graph(directed = True)
    attackGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    attackCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()
    
    
    dayInDistribution = dict()
    dayOutDistribution = dict()
    
    finalDayDistribution = dict()
    
    totalInEdges = 0
    totalOutEdges = 0
    nNodes = 0
    n100 = 0
    n1_10 = 0

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(attackGraph))
    print("")
    
    with open(baseURL+"Dash/AverageShortestPath/AttackGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'AverageShortestPath'])
        
        filewriter.writerow([i + 1, checkGraph.average_path_length(directed=True, unconn=True)])
    
    '''with open(baseURL+"Dash/ClusteringCoefficient/AttackGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'ClusteringCoefficient'])
        
        filewriter.writerow([i + 1, checkGraph.transitivity_undirected()])'''
    
    
    with open(baseURL+"Dash/DiameterValue/AttackGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'Diameter'])
        
        filewriter.writerow([i + 1, checkGraph.diameter(directed=True)])
    
    for vertex in checkGraph.vs():
        totalInEdges += vertex.indegree()
        if vertex.indegree() > 100:
            n100 += 1
        if vertex.indegree() >= 1 and vertex.indegree() <= 10:
            n1_10 += 1
            
        if vertex.indegree() in dayInDistribution:
            dayInDistribution[vertex.indegree()] += 1
            nNodes += 1
        else:
            dayInDistribution[vertex.indegree()] = 1
            nNodes += 1
            
        if vertex.outdegree() in dayOutDistribution:
            dayOutDistribution[vertex.outdegree()] += 1
        else:
            dayOutDistribution[vertex.outdegree()] = 1
    
    print("INDEGREE DISTRIBUTION")
    finalDayDistribution[0] = 0
    finalDayDistribution['1-10'] = 0
    finalDayDistribution['11-20'] = 0
    finalDayDistribution['21-30'] = 0
    finalDayDistribution['31-40'] = 0
    finalDayDistribution['41-50'] = 0
    finalDayDistribution['51-60'] = 0
    finalDayDistribution['61-70'] = 0
    finalDayDistribution['71-80'] = 0
    finalDayDistribution['81-90'] = 0
    finalDayDistribution['91-100'] = 0
    finalDayDistribution['>100'] = 0  
     
    print("NODES: ", nNodes)
    print("EDGES: ", totalInEdges)
    print("MAJOR THAN 100: ", n100)
    print("BETWEEN 1-10: ", n1_10)
    
    startBound = 1
    endBound = 10
    
    finalDayDistribution[0] = dayInDistribution[0]
    
    while endBound <= 100:
        for key in dayInDistribution:
            if key > 100:
                finalDayDistribution[">100"] += dayInDistribution[key]
            elif key >= startBound and key <= endBound:
                    finalDayDistribution[""+str(startBound)+"-"+str(endBound)+""] += dayInDistribution[key]
        endBound += 10
        startBound += 10
        
    finalDayDistribution[">100"] = finalDayDistribution[">100"]/10
    nodesCont = 0
    totalDistribution = 0
    for key in finalDayDistribution:
        nodesCont += int(finalDayDistribution[key])
        print(key, finalDayDistribution[key])
        totalDistribution += finalDayDistribution[key]/len(checkGraph.vs)
    
    print(nodesCont)
    print(totalDistribution)
    print("")
    
    with open(baseURL+"Dash/InDegreeDistribution/GraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'Type', 'Range', 'Value'])
        for key in finalDayDistribution:
            filewriter.writerow([i + 1, 'Attack', key, finalDayDistribution[key]/len(checkGraph.vs)])
        
    with open(baseURL+"Dash/AverageDegree/AttackGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'AverageDegree'])
        
        filewriter.writerow([i + 1, round(len(checkGraph.es)/len(checkGraph.vs), 3)])
    
    print("OUTDEGREE DISTRIBUTION")
    finalDayDistribution[0] = dayOutDistribution[0]
    finalDayDistribution['1-10'] = 0
    finalDayDistribution['11-20'] = 0
    finalDayDistribution['21-30'] = 0
    finalDayDistribution['31-40'] = 0
    finalDayDistribution['41-50'] = 0
    finalDayDistribution['51-60'] = 0
    finalDayDistribution['61-70'] = 0
    finalDayDistribution['71-80'] = 0
    finalDayDistribution['81-90'] = 0
    finalDayDistribution['91-100'] = 0
    finalDayDistribution['>100'] = 0  
    
    startBound = 1
    endBound = 10
    
    while endBound <= 100:
        for key in dayOutDistribution:
            if key > 100:
                finalDayDistribution[">100"] += dayOutDistribution[key]
            elif key >= startBound and key <= endBound:
                    finalDayDistribution[""+str(startBound)+"-"+str(endBound)+""] += dayOutDistribution[key]
        endBound += 10
        startBound += 10
        
        
    finalDayDistribution[">100"] = finalDayDistribution[">100"]/10
    
    nodesCont = 0
    totalDistribution = 0
    for key in finalDayDistribution:
        nodesCont += int(finalDayDistribution[key])
        print(key, finalDayDistribution[key])
        totalDistribution += finalDayDistribution[key]/len(checkGraph.vs)
    
    print(nodesCont)
    print(totalDistribution)
    
    with open(baseURL+"Dash/OutDegreeDistribution/GraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'Type', 'Range', 'Value'])
        for key in finalDayDistribution:
            filewriter.writerow([i + 1, 'Attack', key, finalDayDistribution[key]/len(checkGraph.vs)])      
    
    print("")
        
    
    #for each edge of the multigraph
    for itr in list(attackGraph.edges(None, True, True)):
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                edgesDict[itr[0]][itr[1]] += 1
                attackCount += 1
            else:
                edgesDict[itr[0]][itr[1]] = 1
                attackCount += 1
        else:
            edgesDict[itr[0]] = {}
            edgesDict[itr[0]][itr[1]] = 1
            attackCount += 1

    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            nStep += 1
            compactAttackGraph.add_edge(int(fromVertex), int(toVertex), Weight = edgesDict[fromVertex][toVertex])
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += edgesDict[fromVertex][toVertex]

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(attackGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactAttackGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF ATTACK EDGES MET: ", attackCount)
    nx.write_graphml(compactAttackGraph, baseURL + "Dataset/Graphml-File/Compact/AttackGraph/cAttack-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    

DAY:  1
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2807
Number of edges: 32997
Average in degree:  11.7553
Average out degree:  11.7553

INDEGREE DISTRIBUTION
NODES:  2807
EDGES:  32997
MAJOR THAN 100:  13
BETWEEN 1-10:  1096
0 747
1-10 1096
11-20 408
21-30 247
31-40 120
41-50 79
51-60 48
61-70 26
71-80 11
81-90 7
91-100 5
>100 13.0
2807
1.0

OUTDEGREE DISTRIBUTION
0 1869
1-10 625
11-20 128
21-30 48
31-40 23
41-50 23
51-60 13
61-70 7
71-80 5
81-90 7
91-100 4
>100 55.0
2807
1.0

FROM VERTICES NUMBER IN DICTIONARY:  938
START GRAPH ALL VERTICES:  2807
VERTICES WITH OUTDEGREE 0: 1869

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2807
Number of edges: 7476
Average in degree:   2.6633
Average out degree:   2.6633

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  32997
NUMBER OF STEP (END GRAPH EDGES NUMBER):  7476
NUMBER OF ATTACK EDGES MET:  32997
----------------------------------------------------------------------
DAY:  2
START GRAPH
Name: 
Type: MultiDiGraph
Nu

DAY:  10
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2145
Number of edges: 22778
Average in degree:  10.6191
Average out degree:  10.6191

INDEGREE DISTRIBUTION
NODES:  2145
EDGES:  22778
MAJOR THAN 100:  10
BETWEEN 1-10:  794
0 659
1-10 794
11-20 316
21-30 173
31-40 94
41-50 48
51-60 17
61-70 15
71-80 9
81-90 6
91-100 4
>100 10.0
2145
0.9999999999999999

OUTDEGREE DISTRIBUTION
0 1321
1-10 556
11-20 108
21-30 53
31-40 26
41-50 13
51-60 9
61-70 6
71-80 2
81-90 8
91-100 3
>100 40.0
2145
0.9999999999999998

FROM VERTICES NUMBER IN DICTIONARY:  824
START GRAPH ALL VERTICES:  2145
VERTICES WITH OUTDEGREE 0: 1321

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2145
Number of edges: 5125
Average in degree:   2.3893
Average out degree:   2.3893

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  22778
NUMBER OF STEP (END GRAPH EDGES NUMBER):  5125
NUMBER OF ATTACK EDGES MET:  22778
----------------------------------------------------------------------
DAY:  11
START GRAPH
N

DAY:  19
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1805
Number of edges: 18820
Average in degree:  10.4266
Average out degree:  10.4266

INDEGREE DISTRIBUTION
NODES:  1805
EDGES:  18820
MAJOR THAN 100:  7
BETWEEN 1-10:  694
0 529
1-10 694
11-20 260
21-30 150
31-40 78
41-50 48
51-60 19
61-70 12
71-80 2
81-90 3
91-100 3
>100 7.0
1805
1.0

OUTDEGREE DISTRIBUTION
0 1153
1-10 439
11-20 82
21-30 29
31-40 18
41-50 23
51-60 14
61-70 5
71-80 2
81-90 3
91-100 5
>100 32.0
1805
0.9999999999999999

FROM VERTICES NUMBER IN DICTIONARY:  652
START GRAPH ALL VERTICES:  1805
VERTICES WITH OUTDEGREE 0: 1153

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1805
Number of edges: 4286
Average in degree:   2.3745
Average out degree:   2.3745

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  18820
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4286
NUMBER OF ATTACK EDGES MET:  18820
----------------------------------------------------------------------
DAY:  20
START GRAPH
Name: 
Type: Multi

DAY:  28
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1633
Number of edges: 17057
Average in degree:  10.4452
Average out degree:  10.4452

INDEGREE DISTRIBUTION
NODES:  1633
EDGES:  17057
MAJOR THAN 100:  6
BETWEEN 1-10:  636
0 500
1-10 636
11-20 223
21-30 99
31-40 76
41-50 45
51-60 20
61-70 13
71-80 6
81-90 6
91-100 3
>100 6.0
1633
1.0

OUTDEGREE DISTRIBUTION
0 1007
1-10 431
11-20 71
21-30 29
31-40 16
41-50 15
51-60 10
61-70 16
71-80 8
81-90 1
91-100 1
>100 28.0
1633
1.0

FROM VERTICES NUMBER IN DICTIONARY:  626
START GRAPH ALL VERTICES:  1633
VERTICES WITH OUTDEGREE 0: 1007

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1633
Number of edges: 3905
Average in degree:   2.3913
Average out degree:   2.3913

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  17057
NUMBER OF STEP (END GRAPH EDGES NUMBER):  3905
NUMBER OF ATTACK EDGES MET:  17057
----------------------------------------------------------------------
DAY:  29
START GRAPH
Name: 
Type: MultiDiGraph
Number 

## Create compact trade graph

In [5]:
# Creation of 30 plain graphs
nGraph = 30
compactTradeGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

cont = 0
tradeCount = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    tradeGraph = nx.MultiDiGraph()
    compactTradeGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    tradeGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    tradeCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()
    
    dayInDistribution = dict()
    dayOutDistribution = dict()
    
    finalDayDistribution = dict()
    
    totalInEdges = 0
    totalOutEdges = 0
    nNodes = 0
    n100 = 0
    n1_10 = 0
    
    with open(baseURL+"Dash/AverageShortestPath/TradeGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'AverageShortestPath'])
        
        filewriter.writerow([i + 1, checkGraph.average_path_length(directed=True, unconn=True)])
    
    '''with open(baseURL+"Dash/ClusteringCoefficient/TradeGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'ClusteringCoefficient'])
        
        filewriter.writerow([i + 1, checkGraph.transitivity_undirected()])'''
    
    with open(baseURL+"Dash/DiameterValue/TradeGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'Diameter'])
        
        filewriter.writerow([i + 1, checkGraph.diameter(directed=True)])
    
    for vertex in checkGraph.vs():
        totalInEdges += vertex.indegree()
        if vertex.indegree() > 100:
            n100 += 1
        if vertex.indegree() >= 1 and vertex.indegree() <= 10:
            n1_10 += 1
            
        if vertex.indegree() in dayInDistribution:
            dayInDistribution[vertex.indegree()] += 1
            nNodes += 1
        else:
            dayInDistribution[vertex.indegree()] = 1
            nNodes += 1
            
        if vertex.outdegree() in dayOutDistribution:
            dayOutDistribution[vertex.outdegree()] += 1
        else:
            dayOutDistribution[vertex.outdegree()] = 1
    
    print("INDEGREE DISTRIBUTION")
    finalDayDistribution[0] = 0
    finalDayDistribution['1-10'] = 0
    finalDayDistribution['11-20'] = 0
    finalDayDistribution['21-30'] = 0
    finalDayDistribution['31-40'] = 0
    finalDayDistribution['41-50'] = 0
    finalDayDistribution['51-60'] = 0
    finalDayDistribution['61-70'] = 0
    finalDayDistribution['71-80'] = 0
    finalDayDistribution['81-90'] = 0
    finalDayDistribution['91-100'] = 0
    finalDayDistribution['>100'] = 0  
     
    print("NODES: ", nNodes)
    print("EDGES: ", totalInEdges)
    print("MAJOR THAN 100: ", n100)
    print("BETWEEN 1-10: ", n1_10)
    
    startBound = 1
    endBound = 10
    
    finalDayDistribution[0] = dayInDistribution[0]
    
    while endBound <= 100:
        for key in dayInDistribution:
            if key > 100:
                finalDayDistribution[">100"] += dayInDistribution[key]
            elif key >= startBound and key <= endBound:
                    finalDayDistribution[""+str(startBound)+"-"+str(endBound)+""] += dayInDistribution[key]
        endBound += 10
        startBound += 10
        
    finalDayDistribution[">100"] = finalDayDistribution[">100"]/10
    nodesCont = 0
    totalDistribution = 0
    for key in finalDayDistribution:
        nodesCont += int(finalDayDistribution[key])
        print(key, finalDayDistribution[key])
        totalDistribution += finalDayDistribution[key]/len(checkGraph.vs)
    
    print(nodesCont)
    print(totalDistribution)
    print("")
    
    with open(baseURL+"Dash/InDegreeDistribution/GraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for key in finalDayDistribution:
            filewriter.writerow([i + 1, 'Trade', key, finalDayDistribution[key]/len(checkGraph.vs)])
        
    with open(baseURL+"Dash/AverageDegree/TradeGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'AverageDegree'])
        
        filewriter.writerow([i + 1, round(len(checkGraph.es)/len(checkGraph.vs), 3)])
    
    print("OUTDEGREE DISTRIBUTION")
    finalDayDistribution[0] = dayOutDistribution[0]
    finalDayDistribution['1-10'] = 0
    finalDayDistribution['11-20'] = 0
    finalDayDistribution['21-30'] = 0
    finalDayDistribution['31-40'] = 0
    finalDayDistribution['41-50'] = 0
    finalDayDistribution['51-60'] = 0
    finalDayDistribution['61-70'] = 0
    finalDayDistribution['71-80'] = 0
    finalDayDistribution['81-90'] = 0
    finalDayDistribution['91-100'] = 0
    finalDayDistribution['>100'] = 0  
    
    startBound = 1
    endBound = 10
    
    while endBound <= 100:
        for key in dayOutDistribution:
            if key > 100:
                finalDayDistribution[">100"] += dayOutDistribution[key]
            elif key >= startBound and key <= endBound:
                    finalDayDistribution[""+str(startBound)+"-"+str(endBound)+""] += dayOutDistribution[key]
        endBound += 10
        startBound += 10
        
        
    finalDayDistribution[">100"] = finalDayDistribution[">100"]/10
    
    nodesCont = 0
    totalDistribution = 0
    for key in finalDayDistribution:
        nodesCont += int(finalDayDistribution[key])
        print(key, finalDayDistribution[key])
        totalDistribution += finalDayDistribution[key]/len(checkGraph.vs)
    
    print(nodesCont)
    print(totalDistribution)
    
    
    with open(baseURL+"Dash/OutDegreeDistribution/GraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for key in finalDayDistribution:
            filewriter.writerow([i + 1, 'Trade', key, finalDayDistribution[key]/len(checkGraph.vs)])

    print("")

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(tradeGraph))
    print("")
    
    #for each edge of the multigraph
    for itr in list(tradeGraph.edges(None, True, True)):
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                edgesDict[itr[0]][itr[1]] += 1
                tradeCount += 1
            else:
                edgesDict[itr[0]][itr[1]] = 1
                tradeCount += 1
        else:
            edgesDict[itr[0]] = {}
            edgesDict[itr[0]][itr[1]] = 1
            tradeCount += 1

    
    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            if fromVertex == "2789":
                print(fromVertex)
                cont += edgesDict[fromVertex][toVertex]
            nStep += 1
            compactTradeGraph.add_edge(int(fromVertex), int(toVertex), Weight = edgesDict[fromVertex][toVertex])
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += edgesDict[fromVertex][toVertex]

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(tradeGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactTradeGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    nx.write_graphml(compactTradeGraph, baseURL + "Dataset/Graphml-File/Compact/TradeGraph/cTrade-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')
print(cont)
    

INDEGREE DISTRIBUTION
NODES:  1336
EDGES:  9390
MAJOR THAN 100:  5
BETWEEN 1-10:  873
0 222
1-10 873
11-20 139
21-30 45
31-40 29
41-50 9
51-60 6
61-70 4
71-80 2
81-90 2
91-100 0
>100 5.0
1336
0.9999999999999999

OUTDEGREE DISTRIBUTION
0 127
1-10 962
11-20 155
21-30 45
31-40 22
41-50 8
51-60 5
61-70 4
71-80 1
81-90 3
91-100 1
>100 3.0
1336
0.9999999999999999

DAY:  1
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1336
Number of edges: 9390
Average in degree:   7.0284
Average out degree:   7.0284

2789
FROM VERTICES NUMBER IN DICTIONARY:  1209
START GRAPH ALL VERTICES:  1336
VERTICES WITH OUTDEGREE 0: 127

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1336
Number of edges: 4909
Average in degree:   3.6744
Average out degree:   3.6744

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  9390
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4909
NUMBER OF TRADE EDGES MET:  9390
----------------------------------------------------------------------
INDEGREE DISTRIBUTION
NODES:  13

INDEGREE DISTRIBUTION
NODES:  1248
EDGES:  8921
MAJOR THAN 100:  2
BETWEEN 1-10:  774
0 213
1-10 774
11-20 163
21-30 47
31-40 22
41-50 13
51-60 7
61-70 4
71-80 0
81-90 0
91-100 3
>100 2.0
1248
0.9999999999999999

OUTDEGREE DISTRIBUTION
0 109
1-10 875
11-20 168
21-30 47
31-40 23
41-50 14
51-60 5
61-70 6
71-80 0
81-90 0
91-100 1
>100 0.0
1248
0.9999999999999999

DAY:  10
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1248
Number of edges: 8921
Average in degree:   7.1482
Average out degree:   7.1482

2789
FROM VERTICES NUMBER IN DICTIONARY:  1139
START GRAPH ALL VERTICES:  1248
VERTICES WITH OUTDEGREE 0: 109

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1248
Number of edges: 4769
Average in degree:   3.8213
Average out degree:   3.8213

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  8921
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4769
NUMBER OF TRADE EDGES MET:  8921
----------------------------------------------------------------------
INDEGREE DISTRIBUTION
NODES: 

INDEGREE DISTRIBUTION
NODES:  1099
EDGES:  8624
MAJOR THAN 100:  6
BETWEEN 1-10:  631
0 234
1-10 631
11-20 140
21-30 43
31-40 23
41-50 7
51-60 4
61-70 4
71-80 5
81-90 0
91-100 2
>100 6.0
1099
1.0000000000000002

OUTDEGREE DISTRIBUTION
0 101
1-10 758
11-20 144
21-30 47
31-40 24
41-50 7
51-60 5
61-70 2
71-80 4
81-90 2
91-100 2
>100 3.0
1099
0.9999999999999999

DAY:  19
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1099
Number of edges: 8624
Average in degree:   7.8471
Average out degree:   7.8471

2789
FROM VERTICES NUMBER IN DICTIONARY:  998
START GRAPH ALL VERTICES:  1099
VERTICES WITH OUTDEGREE 0: 101

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1099
Number of edges: 4269
Average in degree:   3.8844
Average out degree:   3.8844

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  8624
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4269
NUMBER OF TRADE EDGES MET:  8624
----------------------------------------------------------------------
INDEGREE DISTRIBUTION
NODES:  10

INDEGREE DISTRIBUTION
NODES:  1017
EDGES:  8665
MAJOR THAN 100:  6
BETWEEN 1-10:  620
0 169
1-10 620
11-20 120
21-30 46
31-40 27
41-50 12
51-60 4
61-70 9
71-80 1
81-90 3
91-100 0
>100 6.0
1017
1.0000000000000002

OUTDEGREE DISTRIBUTION
0 92
1-10 684
11-20 134
21-30 52
31-40 24
41-50 11
51-60 4
61-70 8
71-80 1
81-90 2
91-100 1
>100 4.0
1017
1.0

DAY:  28
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1017
Number of edges: 8665
Average in degree:   8.5202
Average out degree:   8.5202

2789
FROM VERTICES NUMBER IN DICTIONARY:  925
START GRAPH ALL VERTICES:  1017
VERTICES WITH OUTDEGREE 0: 92

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1017
Number of edges: 4057
Average in degree:   3.9892
Average out degree:   3.9892

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  8665
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4057
NUMBER OF TRADE EDGES MET:  8665
----------------------------------------------------------------------
INDEGREE DISTRIBUTION
NODES:  991
EDGES:  8606


## Create compact message graph

In [6]:
# Creation of 30 plain graphs
nGraph = 30
compactMessageGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

messageCount = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    messageGraph = nx.MultiDiGraph()
    compactMessageGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    messageGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    messageCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(messageGraph))
    print("")
    
    with open(baseURL+"Dash/AverageShortestPath/MessageGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'AverageShortestPath'])
        
        filewriter.writerow([i + 1, checkGraph.average_path_length(directed=True, unconn=True)])
    
    
    '''with open(baseURL+"Dash/ClusteringCoefficient/MessageGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'ClusteringCoefficient'])
        
        filewriter.writerow([i + 1, checkGraph.transitivity_undirected()])'''
    
    with open(baseURL+"Dash/DiameterValue/MessageGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'Diameter'])
        
        filewriter.writerow([i + 1, checkGraph.diameter(directed=True)])
    
    dayInDistribution = dict()
    dayOutDistribution = dict()
    
    finalDayDistribution = dict()
    
    totalInEdges = 0
    totalOutEdges = 0
    nNodes = 0
    n100 = 0
    n1_10 = 0
    
    for vertex in checkGraph.vs():
        totalInEdges += vertex.indegree()
        if vertex.indegree() > 100:
            n100 += 1
        if vertex.indegree() >= 1 and vertex.indegree() <= 10:
            n1_10 += 1
            
        if vertex.indegree() in dayInDistribution:
            dayInDistribution[vertex.indegree()] += 1
            nNodes += 1
        else:
            dayInDistribution[vertex.indegree()] = 1
            nNodes += 1
            
        if vertex.outdegree() in dayOutDistribution:
            dayOutDistribution[vertex.outdegree()] += 1
        else:
            dayOutDistribution[vertex.outdegree()] = 1
    
    print("INDEGREE DISTRIBUTION")
    finalDayDistribution[0] = 0
    finalDayDistribution['1-10'] = 0
    finalDayDistribution['11-20'] = 0
    finalDayDistribution['21-30'] = 0
    finalDayDistribution['31-40'] = 0
    finalDayDistribution['41-50'] = 0
    finalDayDistribution['51-60'] = 0
    finalDayDistribution['61-70'] = 0
    finalDayDistribution['71-80'] = 0
    finalDayDistribution['81-90'] = 0
    finalDayDistribution['91-100'] = 0
    finalDayDistribution['>100'] = 0  
     
    print("NODES: ", nNodes)
    print("EDGES: ", totalInEdges)
    print("MAJOR THAN 100: ", n100)
    print("BETWEEN 1-10: ", n1_10)
    
    startBound = 1
    endBound = 10
    
    finalDayDistribution[0] = dayInDistribution[0]
    
    while endBound <= 100:
        for key in dayInDistribution:
            if key > 100:
                finalDayDistribution[">100"] += dayInDistribution[key]
            elif key >= startBound and key <= endBound:
                    finalDayDistribution[""+str(startBound)+"-"+str(endBound)+""] += dayInDistribution[key]
        endBound += 10
        startBound += 10
        
    finalDayDistribution[">100"] = finalDayDistribution[">100"]/10
    nodesCont = 0
    totalDistribution = 0
    for key in finalDayDistribution:
        nodesCont += int(finalDayDistribution[key])
        print(key, finalDayDistribution[key])
        totalDistribution += finalDayDistribution[key]/len(checkGraph.vs)
    
    print(nodesCont)
    print(totalDistribution)
    print("")
    
    with open(baseURL+"Dash/InDegreeDistribution/GraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        for key in finalDayDistribution:
            filewriter.writerow([i + 1, 'Message', key, finalDayDistribution[key]/len(checkGraph.vs)])
        
    with open(baseURL+"Dash/AverageDegree/MessageGraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if i == 0:
            filewriter.writerow(['Day', 'AverageDegree'])
        
        filewriter.writerow([i + 1, round(len(checkGraph.es)/len(checkGraph.vs), 3)])
    
    print("OUTDEGREE DISTRIBUTION")
    finalDayDistribution[0] = dayOutDistribution[0]
    finalDayDistribution['1-10'] = 0
    finalDayDistribution['11-20'] = 0
    finalDayDistribution['21-30'] = 0
    finalDayDistribution['31-40'] = 0
    finalDayDistribution['41-50'] = 0
    finalDayDistribution['51-60'] = 0
    finalDayDistribution['61-70'] = 0
    finalDayDistribution['71-80'] = 0
    finalDayDistribution['81-90'] = 0
    finalDayDistribution['91-100'] = 0
    finalDayDistribution['>100'] = 0  
    
    startBound = 1
    endBound = 10
    
    while endBound <= 100:
        for key in dayOutDistribution:
            if key > 100:
                finalDayDistribution[">100"] += dayOutDistribution[key]
            elif key >= startBound and key <= endBound:
                    finalDayDistribution[""+str(startBound)+"-"+str(endBound)+""] += dayOutDistribution[key]
        endBound += 10
        startBound += 10
        
        
    finalDayDistribution[">100"] = finalDayDistribution[">100"]/10
    
    nodesCont = 0
    totalDistribution = 0
    for key in finalDayDistribution:
        nodesCont += int(finalDayDistribution[key])
        print(key, finalDayDistribution[key])
        totalDistribution += finalDayDistribution[key]/len(checkGraph.vs)
    
    print(nodesCont)
    print(totalDistribution)

    
    
    with open(baseURL+"Dash/OutDegreeDistribution/GraphData.csv", 'a+') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        for key in finalDayDistribution:
            filewriter.writerow([i + 1, 'Message', key, finalDayDistribution[key]/len(checkGraph.vs)])
            
    print("")
    #for each edge of the multigraph
    for itr in list(messageGraph.edges(None, True, True)):
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                edgesDict[itr[0]][itr[1]] += 1
                messageCount += 1
            else:
                edgesDict[itr[0]][itr[1]] = 1
                messageCount += 1
        else:
            edgesDict[itr[0]] = {}
            edgesDict[itr[0]][itr[1]] = 1
            messageCount += 1

    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            nStep += 1
            compactMessageGraph.add_edge(int(fromVertex), int(toVertex), Weight = edgesDict[fromVertex][toVertex])
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += edgesDict[fromVertex][toVertex]

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(messageGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactMessageGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    nx.write_graphml(compactMessageGraph, baseURL + "Dataset/Graphml-File/Compact/MessageGraph/cMessage-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    

DAY:  1
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1963
Number of edges: 16346
Average in degree:   8.3271
Average out degree:   8.3271

INDEGREE DISTRIBUTION
NODES:  1963
EDGES:  16346
MAJOR THAN 100:  3
BETWEEN 1-10:  1438
0 56
1-10 1438
11-20 325
21-30 76
31-40 30
41-50 12
51-60 6
61-70 7
71-80 5
81-90 2
91-100 3
>100 3.0
1963
1.0

OUTDEGREE DISTRIBUTION
0 795
1-10 912
11-20 116
21-30 46
31-40 14
41-50 9
51-60 7
61-70 11
71-80 3
81-90 10
91-100 2
>100 38.0
1963
1.0000000000000002

FROM VERTICES NUMBER IN DICTIONARY:  1168
START GRAPH ALL VERTICES:  1963
VERTICES WITH OUTDEGREE 0: 795

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1963
Number of edges: 6485
Average in degree:   3.3036
Average out degree:   3.3036

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  16346
NUMBER OF STEP (END GRAPH EDGES NUMBER):  6485
NUMBER OF MESSAGE EDGES MET:  16346
----------------------------------------------------------------------
DAY:  2
START GRAPH
Name: 
Type: MultiDiG

----------------------------------------------------------------------
DAY:  10
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1743
Number of edges: 14643
Average in degree:   8.4010
Average out degree:   8.4010

INDEGREE DISTRIBUTION
NODES:  1743
EDGES:  14643
MAJOR THAN 100:  5
BETWEEN 1-10:  1280
0 59
1-10 1280
11-20 247
21-30 82
31-40 30
41-50 20
51-60 7
61-70 7
71-80 4
81-90 1
91-100 1
>100 5.0
1743
1.0000000000000002

OUTDEGREE DISTRIBUTION
0 670
1-10 847
11-20 112
21-30 28
31-40 16
41-50 8
51-60 12
61-70 10
71-80 6
81-90 2
91-100 1
>100 31.0
1743
0.9999999999999999

FROM VERTICES NUMBER IN DICTIONARY:  1073
START GRAPH ALL VERTICES:  1743
VERTICES WITH OUTDEGREE 0: 670

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1743
Number of edges: 5326
Average in degree:   3.0557
Average out degree:   3.0557

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  14643
NUMBER OF STEP (END GRAPH EDGES NUMBER):  5326
NUMBER OF MESSAGE EDGES MET:  14643
-------------------------

FROM VERTICES NUMBER IN DICTIONARY:  918
START GRAPH ALL VERTICES:  1731
VERTICES WITH OUTDEGREE 0: 813

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1731
Number of edges: 5569
Average in degree:   3.2172
Average out degree:   3.2172

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  12888
NUMBER OF STEP (END GRAPH EDGES NUMBER):  5569
NUMBER OF MESSAGE EDGES MET:  12888
----------------------------------------------------------------------
DAY:  19
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1658
Number of edges: 14883
Average in degree:   8.9765
Average out degree:   8.9765

INDEGREE DISTRIBUTION
NODES:  1658
EDGES:  14883
MAJOR THAN 100:  4
BETWEEN 1-10:  1159
0 40
1-10 1159
11-20 307
21-30 77
31-40 42
41-50 8
51-60 10
61-70 6
71-80 2
81-90 3
91-100 0
>100 4.0
1658
0.9999999999999998

OUTDEGREE DISTRIBUTION
0 791
1-10 670
11-20 81
21-30 26
31-40 17
41-50 7
51-60 13
61-70 13
71-80 7
81-90 3
91-100 1
>100 29.0
1658
1.0

FROM VERTICES NUMBER IN DICTIONARY:  867
S

----------------------------------------------------------------------
DAY:  28
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1512
Number of edges: 8869
Average in degree:   5.8657
Average out degree:   5.8657

INDEGREE DISTRIBUTION
NODES:  1512
EDGES:  8869
MAJOR THAN 100:  1
BETWEEN 1-10:  1238
0 55
1-10 1238
11-20 163
21-30 30
31-40 15
41-50 6
51-60 2
61-70 2
71-80 0
81-90 0
91-100 0
>100 1.0
1512
1.0

OUTDEGREE DISTRIBUTION
0 731
1-10 650
11-20 57
21-30 18
31-40 10
41-50 8
51-60 9
61-70 6
71-80 3
81-90 3
91-100 3
>100 14.0
1512
1.0

FROM VERTICES NUMBER IN DICTIONARY:  781
START GRAPH ALL VERTICES:  1512
VERTICES WITH OUTDEGREE 0: 731

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1512
Number of edges: 3669
Average in degree:   2.4266
Average out degree:   2.4266

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  8869
NUMBER OF STEP (END GRAPH EDGES NUMBER):  3669
NUMBER OF MESSAGE EDGES MET:  8869
----------------------------------------------------------------

## Create Total and Alliance Graph

In [0]:
# create a multigraph for all nets type
messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()
attackGraph = nx.MultiDiGraph()

attackSelf = 0
messageSelf = 0

# number of graph (30 in our case)
nGraph = 30

# for every day is needed create total and alliance graph
for i in range(0, nGraph):
    print("Graph number: ", (i + 1))
    
    # for every day is needed reinitialize the "real" graphs (graph with edge label)
    realAttack = nx.MultiDiGraph()
    realMessage = nx.MultiDiGraph()
    realTrade = nx.MultiDiGraph()
    
    attackSelf = 0
    messageSelf = 0
    
    # for every day is needed reinitialize the alliance graph and total graph
    allianceGraph = nx.MultiDiGraph()
    totalGraph = nx.MultiDiGraph()
    
    # read from Dataset/Graphml-File the correct file based on type and day
    attackGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    
    # copy xxxxGraph in realxxxx adding label to edges; keep attentio about key, it must be unique (we use old key + 
    # letter for define edge tipology) 
    for itr in list(messageGraph.edges(None, True, True)):
    #for itr in messageGraph.edges_iter(None, True, True):
        if int(itr[0]) != int(itr[1]):
            realMessage.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"m", Label= "Message", time = itr[3]['edgetime'])
        else:
            messageSelf += 1
    
    #for itr in tradeGraph.edges_iter(None, True, True):
    for itr in list(tradeGraph.edges(None, True, True)):
        realTrade.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"t", Label = "Trade", time = itr[3]['edgetime'])

    #for itr in attackGraph.edges_iter(None, True, True):
    for itr in list(attackGraph.edges(None, True, True)):
        if int(itr[0]) != int(itr[1]):
            realAttack.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"a", Label= "Attack", time = itr[3]['edgetime'])
        else:
            attackSelf += 1
    
    # compose (a union) in one graph the needed graphs 
    allianceGraph = nx.compose(realMessage, realTrade)
    totalGraph = nx.compose(allianceGraph, realAttack)
    # save total graph and alliance graph in the correct file
    nx.write_graphml(totalGraph, baseURL + "Dataset/Graphml-File/Multi/Total-Network-Graphml/Total-2009-12-"+str(i+1)+".graphml")
    nx.write_graphml(allianceGraph, baseURL + "Dataset/Graphml-File/Multi/Alliance-Network-Graphml/Alliance-2009-12-"+str(i+1)+".graphml")

    # print for check, i checked that sum of graphs is the equal to alliance graph and total graph (according to the case)
    # but i think is a better that YOU MAKE THE SAME; ONE MORE CONTROL IS BETTER!
    print("ATTACK GRAPH: ", nx.info(attackGraph))
    print("NO SENSE SELF ATTACK EDGE: ", attackSelf)
    print("MESSAGE GRAPH: ", nx.info(messageGraph))
    print("NO SENSE SELF MESSAGE EDGE: ", messageSelf)
    print("TRADE GRAPH: ", nx.info(tradeGraph))
    print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
    print("TOTAL GRAPH: ", nx.info(totalGraph))
    print("") 

Graph number:  1
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2807
Number of edges: 32997
Average in degree:  11.7553
Average out degree:  11.7553
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1963
Number of edges: 16346
Average in degree:   8.3271
Average out degree:   8.3271
NO SENSE SELF MESSAGE EDGE:  515
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1336
Number of edges: 9390
Average in degree:   7.0284
Average out degree:   7.0284
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2218
Number of edges: 25221
Average in degree:  11.3711
Average out degree:  11.3711
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 58218
Average in degree:  15.4959
Average out degree:  15.4959

Graph number:  2
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2802
Number of edges: 32782
Average in degree:  11.6995
Average out degree:  11.6995
NO SENSE SELF ATTA

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2156
Number of edges: 22162
Average in degree:  10.2792
Average out degree:  10.2792
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1811
Number of edges: 12486
Average in degree:   6.8945
Average out degree:   6.8945
NO SENSE SELF MESSAGE EDGE:  456
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1205
Number of edges: 9008
Average in degree:   7.4755
Average out degree:   7.4755
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2012
Number of edges: 21038
Average in degree:  10.4563
Average out degree:  10.4563
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 3011
Number of edges: 43200
Average in degree:  14.3474
Average out degree:  14.3474

Graph number:  12
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2082
Number of edges: 21628
Average in degree:  10.3881
Average out degree:  10.3881
NO SENSE SELF ATTACK EDGE:  0
MESS

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1828
Number of edges: 18771
Average in degree:  10.2686
Average out degree:  10.2686
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1572
Number of edges: 11405
Average in degree:   7.2551
Average out degree:   7.2551
NO SENSE SELF MESSAGE EDGE:  295
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1010
Number of edges: 8443
Average in degree:   8.3594
Average out degree:   8.3594
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1735
Number of edges: 19553
Average in degree:  11.2697
Average out degree:  11.2697
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2658
Number of edges: 38324
Average in degree:  14.4184
Average out degree:  14.4184

Graph number:  22
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1775
Number of edges: 16970
Average in degree:   9.5606
Average out degree:   9.5606
NO SENSE SELF ATTACK EDGE:  0
MESS

## Transformation of the Alliance graph to compact plain graph with weighted edges

---

Don't run this code if you already have the plain graph in your dataset folder

In [0]:
#Creation of 30 plain graphs
nGraph = 30
realAllianceGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

tradeCount = 0
messageCount = 0
cont = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    allianceGraph = nx.MultiDiGraph()
    realAllianceGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    allianceGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Alliance-Network-Graphml/Alliance-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Alliance-Network-Graphml/Alliance-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    tradeCount = 0
    messageCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(allianceGraph))
    print("")
    
    #for each edge of the multigraph
    for itr in list(allianceGraph.edges(None, True, True)):
        if itr[0] == '7855':
            cont += 1
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]][0] += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]][1] += 1
                    tradeCount += 1
            else:
                
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]] = [1, 0]
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]] = [0, 1]
                    tradeCount += 1
        else:
            edgesDict[itr[0]] = {}
            
            if itr[3]['Label'] == 'Message':
                edgesDict[itr[0]][itr[1]] = [1, 0]
                messageCount += 1
            elif itr[3]['Label'] == 'Trade':
                edgesDict[itr[0]][itr[1]] = [0, 1]
                tradeCount += 1

    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            nStep += 1
            realAllianceGraph.add_edge(int(fromVertex), int(toVertex), MWeight = edgesDict[fromVertex][toVertex][0], TWeight = edgesDict[fromVertex][toVertex][1])
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += sum(edgesDict[fromVertex][toVertex])

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(allianceGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(realAllianceGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    nx.write_graphml(realAllianceGraph, baseURL + "Dataset/Graphml-File/Compact/AllianceGraph/cAlliance-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    print(cont)
    

DAY:  1
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2218
Number of edges: 25221
Average in degree:  11.3711
Average out degree:  11.3711

FROM VERTICES NUMBER IN DICTIONARY:  1707
START GRAPH ALL VERTICES:  2218
VERTICES WITH OUTDEGREE 0: 511

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2218
Number of edges: 11077
Average in degree:   4.9941
Average out degree:   4.9941

NUMBER OF STEP (END GRAPH EDGES NUMBER):  11077
NUMBER OF TRADE EDGES MET:  9390
NUMBER OF MESSAGE EDGES MET:  15831
----------------------------------------------------------------------
2
DAY:  2
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2311
Number of edges: 23310
Average in degree:  10.0865
Average out degree:  10.0865

FROM VERTICES NUMBER IN DICTIONARY:  1659
START GRAPH ALL VERTICES:  2311
VERTICES WITH OUTDEGREE 0: 652

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2311
Number of edges: 10448
Average in degree:   4.5210
Average out degree:   4.5210

NUMBER

----------------------------------------------------------------------
7
DAY:  15
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1948
Number of edges: 21580
Average in degree:  11.0780
Average out degree:  11.0780

FROM VERTICES NUMBER IN DICTIONARY:  1439
START GRAPH ALL VERTICES:  1948
VERTICES WITH OUTDEGREE 0: 509

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1948
Number of edges: 9746
Average in degree:   5.0031
Average out degree:   5.0031

NUMBER OF STEP (END GRAPH EDGES NUMBER):  9746
NUMBER OF TRADE EDGES MET:  10175
NUMBER OF MESSAGE EDGES MET:  11405
----------------------------------------------------------------------
8
DAY:  16
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1925
Number of edges: 23046
Average in degree:  11.9719
Average out degree:  11.9719

FROM VERTICES NUMBER IN DICTIONARY:  1385
START GRAPH ALL VERTICES:  1925
VERTICES WITH OUTDEGREE 0: 540

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1925
Number of edg

----------------------------------------------------------------------
22
DAY:  29
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1595
Number of edges: 15994
Average in degree:  10.0276
Average out degree:  10.0276

FROM VERTICES NUMBER IN DICTIONARY:  1192
START GRAPH ALL VERTICES:  1595
VERTICES WITH OUTDEGREE 0: 403

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1595
Number of edges: 7064
Average in degree:   4.4288
Average out degree:   4.4288

NUMBER OF STEP (END GRAPH EDGES NUMBER):  7064
NUMBER OF TRADE EDGES MET:  8606
NUMBER OF MESSAGE EDGES MET:  7388
----------------------------------------------------------------------
22
DAY:  30
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1670
Number of edges: 16735
Average in degree:  10.0210
Average out degree:  10.0210

FROM VERTICES NUMBER IN DICTIONARY:  1155
START GRAPH ALL VERTICES:  1670
VERTICES WITH OUTDEGREE 0: 515

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1670
Number of edg

## Transformation of the total multigraph to a multigraph with weighted edges for each type of interaction

---

Don't run this code if you already have the plain graph in your dataset folder

In [0]:

nGraph = 30
compactAllGraph = nx.MultiDiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

tradeCount = 0
messageCount = 0
attackCount = 0
finalCheck = 0
cont = 0

#for each day 
for i in range(0, nGraph):
    
    #create new plaingraphs 
    allGraph = nx.MultiDiGraph()
    compactAllGraph = nx.MultiDiGraph()
    checkGraph = ig.Graph()
    allGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Total-Network-Graphml/Total-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Total-Network-Graphml/Total-2009-12-"+str(i+1)+".graphml")
    
    #initialize counters
    tradeCount = 0
    messageCount = 0
    attackCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    finalCheck = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(allGraph))
    print("")

    #create a tmp dictionary to count the numbers of messages, trades and attacks between each couple of nodes 
    for itr in list(allGraph.edges(None, True, True)):
        # print(itr)
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                    
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]][0] += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]][1] += 1
                    tradeCount += 1
                elif itr[3]['Label'] == 'Attack':
                    edgesDict[itr[0]][itr[1]][2] += 1
                    attackCount += 1
            else: 
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]] = [1, 0, 0]
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]] = [0, 1, 0]
                    tradeCount += 1
                elif itr[3]['Label'] == 'Attack':
                    edgesDict[itr[0]][itr[1]] = [0, 0, 1]
                    attackCount += 1
        else:
            edgesDict[itr[0]] = {}
            
            if itr[3]['Label'] == 'Message':
                edgesDict[itr[0]][itr[1]] = [1, 0, 0]
                messageCount += 1
            elif itr[3]['Label'] == 'Trade':
                edgesDict[itr[0]][itr[1]] = [0, 1, 0]
                tradeCount += 1
            elif itr[3]['Label'] == 'Attack':
                edgesDict[itr[0]][itr[1]] = [0, 0, 1]
                attackCount += 1
    if '7855' in edgesDict:
        cont += edgesDict['7855']['3778'][1]
   #For each edges saved in the dictionary add it to the plain graph with the correct label 
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            if edgesDict[fromVertex][toVertex][0] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="m", Label = "Message", Weight = edgesDict[fromVertex][toVertex][0])
                nStep += 1
            if edgesDict[fromVertex][toVertex][1] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="t", Label = "Trade", Weight = edgesDict[fromVertex][toVertex][1])
                nStep += 1
            if edgesDict[fromVertex][toVertex][2] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="a", Label = "Attack", Weight = edgesDict[fromVertex][toVertex][2])
                nStep += 1
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += sum(edgesDict[fromVertex][toVertex])

      
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(allGraph))

    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactAllGraph))
    print("")
    
    #Crazy things of Villa
    for itr in list(compactAllGraph.edges(None, True, True)):
        finalCheck += itr[3]['Weight']
    print("PARANOIC CHECK: ", finalCheck)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    print("NUMBER OF ATTACK EDGES MET: ", attackCount)
    nx.write_graphml(compactAllGraph, baseURL + "Dataset/Graphml-File/Compact/TotalGraph/cTotal-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    print(cont)

DAY:  1
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 58218
Average in degree:  15.4959
Average out degree:  15.4959

FROM VERTICES NUMBER IN DICTIONARY:  1872
START GRAPH ALL VERTICES:  3757
VERTICES WITH OUTDEGREE 0: 1885

END GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 18792
Average in degree:   5.0019
Average out degree:   5.0019

PARANOIC CHECK:  58218
SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  58218
NUMBER OF STEP (END GRAPH EDGES NUMBER):  18792
NUMBER OF TRADE EDGES MET:  9390
NUMBER OF MESSAGE EDGES MET:  15831
NUMBER OF ATTACK EDGES MET:  32997
----------------------------------------------------------------------
2
DAY:  2
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 3758
Number of edges: 56092
Average in degree:  14.9260
Average out degree:  14.9260

FROM VERTICES NUMBER IN DICTIONARY:  1836
START GRAPH ALL VERTICES:  3758
VERTICES WITH OUTDEGREE 0: 1922

END GRAPH


Name: 
Type: MultiDiGraph
Number of nodes: 2965
Number of edges: 15431
Average in degree:   5.2044
Average out degree:   5.2044

PARANOIC CHECK:  44983
SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  44983
NUMBER OF STEP (END GRAPH EDGES NUMBER):  15431
NUMBER OF TRADE EDGES MET:  9310
NUMBER OF MESSAGE EDGES MET:  14045
NUMBER OF ATTACK EDGES MET:  21628
----------------------------------------------------------------------
6
DAY:  13
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2937
Number of edges: 44180
Average in degree:  15.0426
Average out degree:  15.0426

FROM VERTICES NUMBER IN DICTIONARY:  1599
START GRAPH ALL VERTICES:  2937
VERTICES WITH OUTDEGREE 0: 1338

END GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2937
Number of edges: 15199
Average in degree:   5.1750
Average out degree:   5.1750

PARANOIC CHECK:  44180
SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  44180
NUMBER OF STEP (END GRAPH EDGES NUMBER):  15199
NUMBER OF TRA

----------------------------------------------------------------------
16
DAY:  24
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2566
Number of edges: 36612
Average in degree:  14.2681
Average out degree:  14.2681

FROM VERTICES NUMBER IN DICTIONARY:  1373
START GRAPH ALL VERTICES:  2566
VERTICES WITH OUTDEGREE 0: 1193

END GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2566
Number of edges: 11987
Average in degree:   4.6715
Average out degree:   4.6715

PARANOIC CHECK:  36612
SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  36612
NUMBER OF STEP (END GRAPH EDGES NUMBER):  11987
NUMBER OF TRADE EDGES MET:  8659
NUMBER OF MESSAGE EDGES MET:  11486
NUMBER OF ATTACK EDGES MET:  16467
----------------------------------------------------------------------
17
DAY:  25
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2525
Number of edges: 31765
Average in degree:  12.5802
Average out degree:  12.5802

FROM VERTICES NUMBER IN DICTIONARY:  130

# Community reorganization in the period

In [0]:
trueTable = dict()
dayCommunities = open(baseURL +  "Dataset/Communities/StartCommunities/communities-2009-12-1.txt", "r")
dictDayCommunities = dict()
nNodes = 0
nCommunity = 0
communityMember = list()
communityAnalogy = list()
nSameMember = 0
nGraph = 30
newCommunity = 0

for community in dayCommunities:
    communityMember = list()
    community = community.split()
    for node in community:
        nNodes += 1
        communityMember.append(node)
    trueTable[nCommunity] = list(communityMember)
    nCommunity += 1
    
print("------TRUE TABLE-------")
print("{:<39}{:<4}".format("TRUE TABLE IS COMPOSED BY NODES: ", nNodes))
print("{:<39}{:<4}".format("TRUE TABLE IS COMPOSED BY COMMUNUTIES: ", nCommunity))
print("")

for i in range(2, nGraph+1):
    dayCommunities = open(baseURL +  "Dataset/Communities/StartCommunities/communities-2009-12-"+str(i)+".txt", "r")
    nNodes = 0
    nCommunity = 0
    for community in dayCommunities:
        nCommunity += 1
        community = community.split()
        for node in community:
            nNodes += 1
    print("DAY: ", i)
    print("{:<39}{:<4}".format("COMMUN DAY IS COMPOSED BY NODES: ", nNodes))
    print("{:<39}{:<4}".format("COMMUN DAY IS COMPOSED BY COMMUNUTIES: ", nCommunity))
    print("")


for i in range(1, nGraph + 1):
    dayCommunities = open(baseURL +  "Dataset/Communities/StartCommunities/communities-2009-12-"+str(i)+".txt", "r")
    dayOrganizeCommunities = open(baseURL +  "Dataset/Communities/OrganizedCommunities/Ocommunities-2009-12-"+str(i)+".txt","w+")
    
    nNodes = 0
    nCommunity = 0
    nSameMember = 0
    communityMember = list()
    row = 0
    
    if i > 2:
        trueTable = dict()
        trueCommunities = open(baseURL +  "Dataset/Communities/OrganizedCommunities/Ocommunities-2009-12-"+str(i - 1)+".txt", "r")
        for community in trueCommunities:
            communityMember = list()
            community = community.split()
            for node in community:
                nNodes += 1
                communityMember.append(node)
            trueTable[nCommunity] = list(communityMember)
            nCommunity += 1
        
    
    nNodes = 0
    nCommunity = 0
    nSameMember = 0
    communityMember = list()
    dictDayCommunities = dict()
    row = 0
    
    for community in dayCommunities:
        communityMember = list()
        community = community.split()
        for node in community:
            nNodes += 1
            communityMember.append(node)
        dictDayCommunities[nCommunity] = list(communityMember)
        nCommunity += 1
    
    print("DAY: ", i)
    print("{:<39}{:<4}".format("COMMUN DAY IS COMPOSED BY NODES: ", nNodes))
    print("{:<39}{:<4}".format("COMMUN DAY IS COMPOSED BY COMMUNUTIES: ", nCommunity + 1))
    print("")
    
    nNodes = 0
    nCommunity = 0
    nSameMember = 0
    row = 0
    communityMember = [0]*(len(trueTable))
    condition = False
    communityChange = list()
    communityStay = list()
    dissolvedCount = 0
    
    # print(trueTable[2])
    # print(dictDayCommunities[2])
    
    for trueCommunity in trueTable:
        communityMember = [0]*len(trueTable)
        for community in dictDayCommunities:
            for vertexA in trueTable[trueCommunity]:
                for vertexB in dictDayCommunities[community]:
                    if vertexB == "DISSOLVED":
                        break
                    if vertexA == vertexB:
                        nSameMember += 1
                        if i == 5:
                            print("{:<16}{:<3}".format("TRUE COMMUNITY: ", trueCommunity))
                            print("{:<16}{:<3}".format("COMMUNITY: ", community))
                            print("VERTEX A: ", vertexA)
                            print("VERTEX B: ", vertexB)
                        if int(trueCommunity) != int(community + dissolvedCount):
                            communityChange.append(int(community + dissolvedCount))
                        else:
                            communityStay.append(int(community + dissolvedCount))
                                            
            if condition:
                print("-----------"+str(trueCommunity)+" VS "+str(community)+"--------------")
                print("SAME MEMBER: ", nSameMember)
                print("TOTAL MEMBER IN TRUE: ", len(trueTable[trueCommunity]))
                print("TOTAL MEMBER IN NEW DAY: ", len(dictDayCommunities[community]))
                print("TRADE: ", nSameMember/len(trueTable[trueCommunity]))
                
            communityMember[community] = nSameMember/len(trueTable[trueCommunity])
            nSameMember = 0
            condition = False
            
        players = ""
        row = communityMember.index(max(communityMember))
        # row += dissolvedCount
        if i == 5:
            print("COMMUNITY INDEX: ", row)
            print(communityMember)
            print("PRESENT IN THIS COMMUNITY", communityChange)
            print("COMMUNITY STAY: ", communityStay)
            print("MELTED COUNT: ", dissolvedCount)
            print("")

        if trueCommunity not in communityStay:
            players += "DISSOLVED"
            dissolvedCount += 1
            if i == 5:
                print("TRUE COMMUNITY: ", trueCommunity)
                print("NOT PRESENTE IN: ", communityStay)
                print("MELTED COUNT: ", dissolvedCount)
        else:
            if row not in communityStay and len(communityStay) > 0:
                print("ROW: ", row)
                print("Community Stay: ", communityStay)
                row = communityStay[0] - dissolvedCount
                print("REAL ROW: ", row)
            for member in dictDayCommunities[row]:
                if '230' in communityStay:
                    print("HERE MAN")
                    print(dictDayCommunities[row])
                players += str(member)
                players += " "
            dictDayCommunities[row].insert(0, "DONE")
        dayOrganizeCommunities.write(players + "\n")
        communityChange.clear()
        communityStay.clear()
        if i == 5:
            print("")
    for community in dictDayCommunities:
        players = ""
        if "DONE" not in dictDayCommunities[community]:
            for member in dictDayCommunities[community]:
                players += str(member)
                players += " "
            dayOrganizeCommunities.write(players + "\n")
    dayOrganizeCommunities.close()
    
    for community in dictDayCommunities:
        if "DONE" not in dictDayCommunities[community]:
            print("New Communities: ", dictDayCommunities[community])
            newCommunity += 1

    
print("NEW COMMUNITY ADDED: ", newCommunity)
        
    
    
                    

    

KeyboardInterrupt: ignored